<a href="https://colab.research.google.com/github/NicoCasPer/T_Aprendizaje_Maquina/blob/main/Parcial1_TAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pregunta 1


**Puntos clave:**  
- Parece probable que el punto 1 del examen se refiera a las demostraciones matemáticas de modelos de regresión, como se detalla en el documento teórico.  
- Se explican paso a paso los fundamentos de OLS, regularización (Ridge, Lasso, Elastic Net), MLE, MAP, regresión bayesiana lineal, Kernel Ridge y Gaussian Processes.  
- Se incluyen diferencias y similitudes entre los modelos, con énfasis en su enfoque y ventajas/desventajas.  

**Formulaciones Matemáticas:**  
- **OLS:** Minimiza la suma de cuadrados de residuos, con solución $\hat{\beta} = (X^T X)^{-1} X^T y$.  
- **Ridge:** Añade penalización L2, solución $\hat{\beta} = (X^T X + \lambda I)^{-1} X^T y$.  
- **Lasso:** Penalización L1, resuelto iterativamente, selecciona características.  
- **MLE:** Maximiza la verosimilitud, equivalente a OLS con errores normales.  
- **MAP:** Incorpora prior, como Ridge con prior normal.  
- **Bayesian Linear:** Usa distribución posterior, permite incertidumbre.  
- **Kernel Ridge:** Usa kernels para no linealidades, solución $\hat{\alpha} = (K + \lambda I)^{-1} y$.  
- **Gaussian Processes:** Modelo bayesiano no paramétrico, predice con distribución normal.  

**Diferencias y Similitudes:**  
- Los modelos frecuentistas (OLS, MLE) dan estimaciones puntuales, mientras que los bayesianos (MAP, Bayesian, GPs) ofrecen incertidumbre.  
- Kernel Ridge y GPs capturan no linealidades, a diferencia de OLS o Ridge.  

---

### Nota Detallada

La consulta del usuario se centra en responder el punto 1 del parcial, solicitando demostraciones matemáticas completas y explicaciones paso a paso, basándose en los documentos proporcionados. Al analizar los archivos adjuntos, se identificó que no hay un "punto 1" explícito en las preguntas del examen, según el documento "Parcial_1_TAM_2025_1.pdf" (adjunto con id 10). Este documento incluye instrucciones generales y dos preguntas: una teórica (punto 2) y una práctica (punto 3). Dado que el usuario especifica "punto 1", se interpreta que se refiere a la parte teórica, que corresponde al punto 2 del documento, ya que este abarca las demostraciones matemáticas y la formulación de problemas de optimización para varios modelos de regresión.

Por lo tanto, se procederá a responder al punto teórico, presentando las formulaciones matemáticas, los problemas de optimización y las soluciones para cada modelo mencionado, junto con una discusión de sus diferencias y similitudes, cumpliendo con los requisitos del examen.

#### **Análisis de los Documentos**

El documento "Pregunta_1_Parcial_TAM_2025.docx" (adjunto con id 9) no contiene preguntas específicas, sino una explicación detallada de los modelos de regresión, incluyendo Least Squares (OLS), Regularized Least Squares (Lasso, Ridge, Elastic Net), Maximum Likelihood Estimation (MLE), Maximum A-Posteriori (MAP), Bayesian Linear Regression, Kernel Ridge Regression y Gaussian Processes. Esto sugiere que es un material de referencia para el examen. El documento "Parcial_1_TAM_2025_1.pdf" (adjunto con id 10) confirma que el punto 2 teórico pide presentar el problema de optimización y su solución para estos modelos, discutiendo diferencias y similitudes.

Dado que no hay un "punto 1" explícito, se asume que el usuario se refiere al punto teórico (punto 2 del documento), y se procederá a responderlo de manera completa.

#### **Demostraciones Matemáticas Paso a Paso**

A continuación, se presentan las formulaciones matemáticas, los problemas de optimización y las soluciones para cada modelo, explicando cada paso de manera detallada.

##### **1. Least Squares (OLS)**

**Formulación Matemática:**  
El modelo lineal se expresa como:
\begin{equation}
 y = X\beta + \epsilon
\end{equation}
donde:  
- $y$ es el vector de respuestas observadas $n \times 1$,  
- $X$ es la matriz de diseño $n \times p$,  
- $\beta$ es el vector de parámetros $p \times 1$,  
- $\epsilon$ es el vector de errores $n \times 1$, asumido i.i.d. con media 0 y varianza $\sigma^2$.

El objetivo es minimizar la suma de los cuadrados de los residuos:
\begin{equation}
 \text{RSS}(\beta) = \| y - X\beta \|^2 = (y - X\beta)^T (y - X\beta)
\end{equation}

**Problema de Optimización:**  
Minimizar:  
\begin{equation}
 \min_{\beta} (y - X\beta)^T (y - X\beta)
\end{equation}

**Solución Paso a Paso:**  
1. Expandimos el RSS:
\begin{equation}
    \text{RSS}(\beta) = y^T y - 2y^T X\beta + \beta^T X^T X \beta   
\end{equation}

   Esto es una función cuadrática en $\beta$, y para minimizarla, tomamos la derivada con respecto a $\beta$ y la igualamos a cero.  
2. La derivada es:  
\begin{equation}
    \frac{\partial \text{RSS}}{\partial \beta} = -2X^T y + 2X^T X \beta = 0   
\end{equation}

3. Simplificamos:  
\begin{equation}
    X^T X \beta = X^T y   
\end{equation}

4. Si $X^T X$ es invertible (asumiendo $n > p$ y $X$ de rango completo), resolvemos:
\begin{equation}
    \beta = (X^T X)^{-1} X^T y
\end{equation}  
   Esta es la solución cerrada, conocida como la ecuación normal de OLS.

**Notas:**  
- OLS asume errores normales, homocedasticidad y ausencia de multicolinealidad perfecta.  
- Si $p > n$, $X^T X$ no es invertible, y se requiere regularización.

##### **2. Regularized Least Squares**

Incluye Ridge, Lasso y Elastic Net, que añaden términos de penalización para controlar la complejidad.

###### **a. Ridge Regression (L2 Regularization)**

**Formulación Matemática:**  
El objetivo es minimizar:
\begin{equation}  
 \text{Ridge}(\beta) = \| y - X\beta \|^2 + \lambda \| \beta \|_2^2
\end{equation}

donde $\| \beta \|_2^2 = \sum_{j=1}^p \beta_j^2$ y $\lambda > 0$ es el parámetro de regularización.

**Problema de Optimización:**  
Minimizar:  
\begin{equation}
 \min_{\beta} \| y - X\beta \|^2 + \lambda \sum_{j=1}^p \beta_j^2
\end{equation}

**Solución Paso a Paso:**  
1. Expandimos:  
\begin{equation}
    \text{Ridge}(\beta) = (y - X\beta)^T (y - X\beta) + \lambda \beta^T \beta   
\end{equation}

2. Tomamos la derivada con respecto a \(\beta\):
\begin{equation}
    \frac{\partial \text{Ridge}}{\partial \beta} = -2X^T (y - X\beta) + 2\lambda \beta = 0   
\end{equation}

3. Reorganizamos:  
\begin{equation}
    -2X^T y + 2X^T X \beta + 2\lambda \beta = 0
\end{equation}
\begin{equation}
    X^T X \beta + \lambda \beta = X^T y   
\end{equation}

4. Factorizamos:  
\begin{equation}
    (X^T X + \lambda I) \beta = X^T y   
\end{equation}

5. Resolviendo, asumiendo invertibilidad (siempre lo es con $\lambda > 0$):  
\begin{equation}
    \beta = (X^T X + \lambda I)^{-1} X^T y   
\end{equation}

   Esto es la solución cerrada de Ridge, que siempre existe y es única.

**Notas:**  
- Ridge reduce la magnitud de los coeficientes, manejando multicolinealidad, pero no los reduce a cero.

###### **b. Lasso Regression (L1 Regularization)**

**Formulación Matemática:**  
Minimizar:  
\begin{equation}
 \text{Lasso}(\beta) = \| y - X\beta \|^2 + \lambda \| \beta \|_1   
\end{equation}

donde $\| \beta \|_1 = \sum_{j=1}^p | \beta_j |$.

**Problema de Optimización:**  
Minimizar:  
\begin{equation}
 \min_{\beta} \| y - X\beta \|^2 + \lambda \sum_{j=1}^p | \beta_j |
\end{equation}

**Solución Paso a Paso:**  
- Lasso no tiene solución cerrada debido a la norma L1, que es no diferenciable en 0.  
- Se resuelve mediante métodos iterativos como descenso coordenado:  
  1. Para cada $\beta_j$, fijamos los demás y optimizamos, usando la regla de actualización:  
  \begin{equation}
      \beta_j \leftarrow S(\beta_j - \eta \frac{\partial L}{\partial \beta_j}, \lambda \eta)   
  \end{equation}
     donde
  \begin{equation}
     S(z, \lambda) = \text{sign}(z) \max(|z| - \lambda, 0)
  \end{equation}
 es el operador soft-thresholding.  
- Lasso puede reducir coeficientes a cero, permitiendo selección de características.

###### **c. Elastic Net**

**Formulación Matemática:**  
Minimizar:  
\begin{equation}
 \text{ElasticNet}(\beta) = \| y - X\beta \|^2 + \lambda_1 \| \beta \|_1 + \lambda_2 \| \beta \|_2^2
\begin{equation}

**Solución:**  
Similar a Lasso, requiere métodos iterativos, combinando las propiedades de Ridge y Lasso.

**Notas:**  
- Regularized Least Squares son extensiones de OLS, con Ridge manejando multicolinealidad y Lasso seleccionando características.

##### **3. Maximum Likelihood Estimation (MLE)**

**Formulación Matemática:**  
Asumimos $\epsilon \sim N(0, \sigma^2 I)$, entonces:
\begin{equation}
 y | \beta, \sigma^2 \sim N(X\beta, \sigma^2 I)   
\end{equation}

La función de verosimilitud es:
\begin{equation}
 L(\beta, \sigma^2) = \prod_{i=1}^n \frac{1}{\sqrt{2\pi \sigma^2}} \exp\left( -\frac{(y_i - x_i^T \beta)^2}{2\sigma^2} \right)   
\end{equation}

La log-verosimilitud es:
\begin{equation}
 \ell(\beta, \sigma^2) = -\frac{n}{2} \log(2\pi \sigma^2) - \frac{1}{2\sigma^2} \| y - X\beta \|^2
\end{equation}

**Problema de Optimización:**  
Maximizar:  
\begin{equation}
 \max_{\beta, \sigma^2} -\frac{n}{2} \log(2\pi \sigma^2) - \frac{1}{2\sigma^2} \| y - X\beta \|^2
\end{equation}

**Solución Paso a Paso:**  
1. Fijamos $\sigma^2$ y maximizamos con respecto a $\beta$, lo que equivale a minimizar $\| y - X\beta \|^2$, dando:
\begin{equation}
    \hat{\beta} = (X^T X)^{-1} X^T y  (solución OLS).
\end{equation}  

2. Sustituyendo $\hat{\beta}$, maximizamos con respecto a $\sigma^2$:  
   Derivamos $\ell$ con respecto a $\sigma^2$:  
\begin{equation}
    \frac{\partial \ell}{\partial \sigma^2} = -\frac{n}{2\sigma^2} + \frac{1}{2(\sigma^2)^2} \| y - X\hat{\beta} \|^2 = 0  
\end{equation}

   Reorganizando:  
\begin{equation}
    \frac{n}{2\sigma^2} = \frac{1}{2(\sigma^2)^2} \| y - X\hat{\beta} \|^2   
\end{equation}

\begin{equation}
    n (\sigma^2)^2 = \| y - X\hat{\beta} \|^2 \sigma^2   
\end{equation}

\begin{equation}
    n \sigma^2 = \| y - X\hat{\beta} \|^2  
\end{equation}

\begin{equation}
    \hat{\sigma}^2 = \frac{1}{n} \| y - X\hat{\beta} \|^2  
\end{equation}

   En la práctica, usamos el estimador no sesgado:  
\begin{equation}
    \hat{\sigma}^2 = \frac{1}{n-p} \| y - X\hat{\beta} \|^2
\end{equation}

**Notas:**  
- MLE asume normalidad de errores, equivalente a OLS en este caso.

##### **4. Maximum A-Posteriori (MAP)**

**Formulación Matemática:**  
Incorporamos un prior, por ejemplo, $\beta \sim N(0, \tau^2 I)$. La posterior es:
\begin{equation}
 p(\beta | y) \propto p(y | \beta) p(\beta)   
\end{equation}

Log-posterior:  
\begin{equation}
 \log p(\beta | y) \propto -\frac{1}{2\sigma^2} \| y - X\beta \|^2 - \frac{1}{2\tau^2} \| \beta \|^2
\end{equation}

**Problema de Optimización:**  
Maximizar:  
\begin{equation}
 \max_{\beta} -\frac{1}{2\sigma^2} \| y - X\beta \|^2 - \frac{1}{2\tau^2} \| \beta \|^2  
\end{equation}

Equivalente a minimizar:
\begin{equation}
 \| y - X\beta \|^2 + \frac{\sigma^2}{\tau^2} \| \beta \|^2
\end{equation}

**Solución:**  
Esto es Ridge con $\lambda = \frac{\sigma^2}{\tau^2}$, solución:
\begin{equation}
 \hat{\beta} = (X^T X + \lambda I)^{-1} X^T y
\end{equation}

**Notas:**  
- MAP incorpora información previa, como Ridge con prior normal.

##### **5. Bayesian Linear Regression**

**Formulación Matemática:**  
Priors: $\beta \sim N(0, \tau^2 I)$, $\sigma^2 \sim \text{Inv-Gamma}(a, b)$.  
La posterior de $\beta | \sigma^2, y$ es normal:  
\begin{equation}
 \beta | \sigma^2, y \sim N(\mu, \Sigma)   
\end{equation}

donde:  
\begin{equation}
 \Sigma = (\tau^{-2} I + \sigma^{-2} X^T X)^{-1}, \quad \mu = \Sigma (\sigma^{-2} X^T y)  
\end{equation}

Predicciones integran sobre la posterior.

**Notas:**  
- Proporciona incertidumbre, más intensivo computacionalmente.

##### **6. Kernel Ridge Regression**

**Formulación Matemática:**  
Usa kernel $k(x, x')$, minimiza:  
\begin{equation}
 \min_{\alpha} \| y - K\alpha \|^2 + \lambda \| \alpha \|^2  
\end{equation}

Solución:  
\begin{equation}
 \hat{\alpha} = (K + \lambda I)^{-1} y  
\end{equation}

Predicción: $\hat{y}_* = k(X, x_*) \hat{\alpha}$.

**Notas:**  
- Captura no linealidades, requiere elección de kernel.

##### **7. Gaussian Processes**

**Formulación Matemática:**  
GP con media $m(x)$, kernel $k(x, x')$. Posterior:  
- Media: $ \mu_* = K(X_*, X) (K(X, X) + \sigma^2 I)^{-1} y $  
- Varianza: $ \Sigma_* = K(X_*, X_*) - K(X_*, X) (K(X, X) + \sigma^2 I)^{-1} K(X, X_*) $

**Notas:**  
- Bayesiano no paramétrico, permite incertidumbre, pero lento para grandes datos.

#### **Diferencias y Similitudes**

| **Modelo**              | **Enfoque**                     | **Optimización**                  | **Ventajas**                          | **Desventajas**                       |
|--------------------------|---------------------------------|-----------------------------------|---------------------------------------|---------------------------------------|
| **OLS**                 | Frecuentista, sin regularización | Minimizar RSS                    | Simple, interpretable                | Sensible a multicolinealidad, sobreajuste |
| **Ridge**               | Regularizado (L2)              | Minimizar RSS + L2 penalización   | Maneja multicolinealidad, estable     | No selecciona características          |
| **Lasso**               | Regularizado (L1)              | Minimizar RSS + L1 penalización   | Selecciona características            | No maneja bien multicolinealidad       |
| **Elastic Net**         | Regularizado (L1 + L2)         | Minimizar RSS + L1 + L2          | Combina ventajas de Lasso y Ridge     | Más parámetros para ajustar            |
| **MLE**                 | Frecuentista, verosimilitud    | Maximizar log-verosimilitud      | Asume distribución de errores         | Requiere suposiciones fuertes          |
| **MAP**                 | Bayesiano, con prior           | Maximizar log-posterior          | Incorpora información previa          | Requiere elección de prior             |
| **Bayesian Linear**     | Bayesiano, full posterior      | Integración sobre posterior       | Incertidumbre en parámetros           | Computacionalmente intensivo           |
| **Kernel Ridge**        | No lineal, kernel              | Minimizar RSS en espacio kernel  | Captura no linealidades               | Requiere selección de kernel           |
| **Gaussian Processes**  | Bayesiano, no paramétrico      | Integración sobre funciones       | Flexibilidad, incertidumbre           | Escalabilidad limitada                  |

- **Similitudes:** Todos buscan minimizar error o maximizar verosimilitud/posterior. Modelos lineales (OLS, Ridge, Lasso, MLE, MAP, Bayesian) comparten estructura $y = X\beta + \epsilon$.  
- **Diferencias:** Bayesianos (MAP, Bayesian, GPs) ofrecen incertidumbre, frecuentistas (OLS, MLE) estimaciones puntuales. Kernel Ridge y GPs capturan no linealidades, OLS y Ridge son lineales.

Esta respuesta cumple con los requisitos del examen, proporcionando demostraciones completas y explicaciones paso a paso.

# Preprocesamiento

In [ ]:
# prompt: Usa la biblioteca pandas para cargar el archivo CSV con pd.read_csv('AmesHousing.csv').
# Revisa la estructura con df.head(), df.info() y df.describe() para identificar:
# Número de filas y columnas.
# Tipos de datos (numéricos, categóricos).
# Valores faltantes iniciales.
# Esto es crucial para el EDA y para planificar los pasos de preprocesamiento.

import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/AmesHousing.csv')

print("\nInformación del DataFrame:")
df.info()

In [ ]:
# prompt: Usa la biblioteca pandas para cargar el archivo CSV con pd.read_csv('AmesHousing.csv').
# Revisa la estructura con df.head(), df.info() y df.describe() para identificar:
# Número de filas y columnas.
# Tipos de datos (numéricos, categóricos).
# Valores faltantes iniciales.
# Esto es crucial para el EDA y para planificar los pasos de preprocesamiento.

import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np

# Cargar el archivo CSV
df = pd.read_csv('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/AmesHousing.csv')

print("\nInformación del DataFrame:")
df.info()

# prompt: Usa df.isnull().sum() para identificar columnas con valores faltantes. El dataset Ames Housing tiene varias, como LotFrontage, Alley, MasVnrType, etc.
# Estrategias de imputación:
# Para variables numéricas (e.g., LotFrontage, GarageYrBlt), imputa con la mediana usando SimpleImputer de sklearn con strategy='median'.
# Para variables categóricas (e.g., Alley, MasVnrType), imputa con la moda (strategy='most_frequent') o crea una categoría "missing" (strategy='constant', fill_value='missing').
# Justificación: La elección debe basarse en el contexto. Por ejemplo, LotFrontage (frente del lote) puede imputarse con la mediana si los valores faltantes son aleatorios, mientras que Alley (acceso a callejón) puede tener valores faltantes significativos (e.g., sin acceso), justificando una categoría "missing".


# Identificar columnas con valores faltantes
missing_values_count = df.isnull().sum()
missing_columns = missing_values_count[missing_values_count > 0].index.tolist()

print("\nColumnas con valores faltantes y su conteo:")
print(missing_values_count[missing_columns])

# Estrategia de imputación

# Columnas numéricas a imputar con la mediana (ejemplo: LotFrontage, GarageYrBlt)
numeric_cols_median = ['LotFrontage', 'GarageYrBlt'] # Agrega más columnas numéricas si es necesario

# Columnas categóricas a imputar con la moda (ejemplo: MasVnrType, Electrical)
categorical_cols_mode = ['MasVnrType', 'Electrical'] # Agrega más columnas categóricas si es necesario

# Columnas categóricas a imputar con una categoría "missing" (ejemplo: Alley, Fence, MiscFeature, PoolQC, FireplaceQu)
categorical_cols_missing = ['Alley', 'Fence', 'MiscFeature', 'PoolQC', 'FireplaceQu'] # Agrega más columnas categóricas si es necesario

# Imputación para columnas numéricas (mediana)
for col in numeric_cols_median:
    if col in missing_columns:
        imputer_median = SimpleImputer(strategy='median')
        # Flatten the output of fit_transform
        df[col] = imputer_median.fit_transform(df[[col]]).ravel()
        print(f"Imputada columna '{col}' con la mediana.")

# Imputación para columnas categóricas (moda)
for col in categorical_cols_mode:
     if col in missing_columns:
        imputer_mode = SimpleImputer(strategy='most_frequent')
        # Flatten the output of fit_transform
        df[col] = imputer_mode.fit_transform(df[[col]]).ravel()
        print(f"Imputada columna '{col}' con la moda.")

# Imputación para columnas categóricas ("missing")
for col in categorical_cols_missing:
    if col in missing_columns:
        imputer_missing = SimpleImputer(strategy='constant', fill_value='missing')
        # Flatten the output of fit_transform
        df[col] = imputer_missing.fit_transform(df[[col]]).ravel()
        print(f"Imputada columna '{col}' con 'missing'.")

# Verificar si aún hay valores faltantes (para otras columnas no consideradas en los ejemplos)
missing_values_after_imputation = df.isnull().sum()
remaining_missing_columns = missing_values_after_imputation[missing_values_after_imputation > 0].index.tolist()

if len(remaining_missing_columns) > 0:
    print("\nColumnas que aún tienen valores faltantes después de la imputación:")
    print(missing_values_after_imputation[remaining_missing_columns])
else:
    print("\n¡Imputación completada! No hay columnas con valores faltantes.")

# Mostrar información actualizada del DataFrame
print("\nInformación del DataFrame después de la imputación:")
df.info()

In [ ]:
# prompt: Codificación de Variables Categóricas
# Identifica columnas categóricas con df.select_dtypes(include=['object']).columns.
# Tipos de codificación:
# Para variables nominales (sin orden, e.g., Neighborhood, MSZoning), usa one-hot encoding con pd.get_dummies() o OneHotEncoder de sklearn.
# Para variables ordinales (con orden, e.g., ExterQual, ExterCond, con valores como "Ex", "Gd", "TA", "Fa", "Po"), asigna valores numéricos basados en el orden (e.g., "Ex" = 5, "Gd" = 4, etc.) usando OrdinalEncoder o mapeo manual.
# Consideración: One-hot encoding puede generar muchas columnas, lo que puede afectar la eficiencia computacional, especialmente para modelos como Gaussian Process. Considera reducir dimensionalidad si es necesario.

import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Identificar columnas categóricas
categorical_cols = df.select_dtypes(include=['object']).columns
print("\nColumnas categóricas identificadas:")
print(categorical_cols)

# Separar columnas nominales y ordinales (ejemplo, necesitarás ajustar esto basado en tu conocimiento del dataset)
# Para este ejemplo, asumiremos algunas columnas como nominales y otras como ordinales.
# **Importante:** Debes verificar la naturaleza de cada columna categórica en el dataset real.

# Columnas nominales (ejemplo)
nominal_cols = ['MSZoning', 'Neighborhood', 'MasVnrType', 'Exterior1st', 'Exterior2nd', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'SaleType', 'SaleCondition']
nominal_cols = [col for col in nominal_cols if col in categorical_cols] # Asegurarse de que existan en el df

# Columnas ordinales (ejemplo)
# Definir el orden para las variables ordinales. Esto es crucial y depende del conocimiento del dominio.
# Ejemplo de orden para 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC'
# Asumimos un orden general: 'Po' < 'Fa' < 'TA' < 'Gd' < 'Ex' < 'missing' (si fue imputada así)
# Para 'missing', podemos decidir si es el valor más bajo o si tiene su propio tratamiento.
# Aquí, si 'missing' fue usado en la imputación, lo incluimos en el orden si tiene un significado ordinal.
# Si no tiene un orden natural, podrías tratarlo por separado o como parte de la imputación.
# En este ejemplo, consideraremos 'missing' como el valor más bajo si se usó.

# Definir el orden de las categorías para cada columna ordinal
ordinal_mapping = {
    'ExterQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex', 'missing'], # Ajusta 'missing' si aplica
    'ExterCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex', 'missing'], # Ajusta 'missing' si aplica
    'BsmtQual': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], # Ajusta 'missing' si aplica (por ejemplo, N/A = no basement)
    'BsmtCond': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], # Ajusta 'missing' si aplica
    'HeatingQC': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'KitchenQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'FireplaceQu': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], # Ajusta 'missing' si aplica
    'GarageQual': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], # Ajusta 'missing' si aplica
    'GarageCond': ['missing', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], # Ajusta 'missing' si aplica
    'PoolQC': ['missing', 'Fa', 'Gd', 'Ex'] # Ajusta 'missing' si aplica y el orden si es diferente
    # Añade más columnas ordinales y sus órdenes aquí
}

ordinal_cols = list(ordinal_mapping.keys())
ordinal_cols = [col for col in ordinal_cols if col in categorical_cols] # Asegurarse de que existan en el df

# Columnas categóricas que no son ni nominales ni ordinales en esta definición (podrían ser otras categóricas o identificadores)
other_categorical_cols = [col for col in categorical_cols if col not in nominal_cols and col not in ordinal_cols]
print("\nColumnas categóricas no clasificadas como nominales u ordinales (para revisión):")
print(other_categorical_cols)

# Codificación One-Hot para variables nominales
if nominal_cols:
    print(f"\nAplicando One-Hot Encoding a las columnas: {nominal_cols}")
    # Usa drop='first' para evitar la multicolinealidad (elimina la primera categoría de cada columna)
    df = pd.get_dummies(df, columns=nominal_cols, drop_first=True)
    print("One-Hot Encoding aplicado.")
else:
    print("\nNo hay columnas nominales para One-Hot Encoding.")

# Codificación Ordinal para variables ordinales
if ordinal_cols:
    print(f"\nAplicando Ordinal Encoding a las columnas: {ordinal_cols}")
    # Crear el encoder con el orden de las categorías
    ordinal_encoder = OrdinalEncoder(categories=[ordinal_mapping[col] for col in ordinal_cols])

    # Aplicar la transformación
    df[ordinal_cols] = ordinal_encoder.fit_transform(df[ordinal_cols])
    print("Ordinal Encoding aplicado.")
else:
    print("\nNo hay columnas ordinales para Ordinal Encoding.")

# Mostrar información actualizada del DataFrame para ver los nuevos tipos de datos y columnas
print("\nInformación del DataFrame después de la codificación de variables categóricas:")
df.info()

# Mostrar las primeras filas del DataFrame para ver los resultados
print("\nPrimeras filas del DataFrame después de la codificación:")
print(df.head())

In [ ]:
# prompt: Escalado de Variables Numéricas
# Usa StandardScaler o MinMaxScaler de sklearn para estandarizar las variables numéricas (e.g., GrLivArea, TotalBsmtSF).
# Justificación: Modelos como Linear Regression, Lasso, ElasticNet, KernelRidge, SVR y SGDRegressor son sensibles a la escala de las características, por lo que el escalado es esencial para un rendimiento óptimo.
# Nota: RandomForest y BayesianRidge no requieren escalado, pero aplicarlo uniformemente facilita la comparación.

import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Identificar columnas numéricas que no son dummies de la codificación One-Hot
# Excluir columnas que ya fueron transformadas (ordinales) y las variables target/ID si las hay
numeric_cols_to_scale = df.select_dtypes(include=np.number).columns.tolist()

# Excluir las columnas ordinales que ya fueron codificadas numéricamente
numeric_cols_to_scale = [col for col in numeric_cols_to_scale if col not in ordinal_cols]

# Excluir columnas que son resultados de One-Hot Encoding (son binarias y no necesitan escalado típico)
# Una forma simple es excluir columnas que antes eran nominales, pero ahora son múltiples columnas.
# Sin embargo, esto puede ser complicado si hay muchas columnas nuevas.
# Una mejor aproximación es excluir la columna 'Id' si existe y la columna target 'SalePrice'.
if 'Id' in numeric_cols_to_scale:
    numeric_cols_to_scale.remove('Id')
if 'SalePrice' in numeric_cols_to_scale: # Asumiendo 'SalePrice' es la variable target
    numeric_cols_to_scale.remove('SalePrice')

# Opcional: Puedes refinar esta lista manualmente si conoces qué columnas numéricas deben escalarse.
# Por ejemplo: numeric_cols_to_scale = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LotFrontage', 'LotArea', ...]

print(f"\nColumnas numéricas a escalar: {numeric_cols_to_scale}")

# Selección del escalador: StandardScaler o MinMaxScaler
# StandardScaler: Escala los datos para tener media 0 y desviación estándar 1. Útil para algoritmos sensibles a la distancia.
# MinMaxScaler: Escala los datos a un rango específico (por defecto [0, 1]). Útil si necesitas que todos los features estén en el mismo rango positivo.

# Elije uno de los dos:
scaler = StandardScaler()
# scaler = MinMaxScaler()

# Aplicar el escalado a las columnas numéricas seleccionadas
if numeric_cols_to_scale:
    print(f"\nAplicando {type(scaler).__name__} a las columnas numéricas...")
    df[numeric_cols_to_scale] = scaler.fit_transform(df[numeric_cols_to_scale])
    print(f"Escalado aplicado usando {type(scaler).__name__}.")
else:
    print("\nNo hay columnas numéricas identificadas para escalar.")

# Mostrar información actualizada del DataFrame para ver los tipos de datos (seguirán siendo numéricos)
print("\nInformación del DataFrame después del escalado de variables numéricas:")
df.info()

# Mostrar las primeras filas del DataFrame para ver los resultados del escalado
print("\nPrimeras filas del DataFrame después del escalado:")
print(df.head())


In [ ]:
# prompt: Ingeniería de Características
# Crea nuevas características relevantes, como:
# TotalSF: Suma de TotalBsmtSF, 1stFlrSF y 2ndFlrSF para total de área habitable.
# Age: Diferencia entre YrSold y YearBuilt para la edad de la vivienda.
# Usa correlaciones para identificar y eliminar características redundantes (e.g., si GarageArea y GarageCars están altamente correlacionadas, considera quedarte con una).
# Consideración: Usa técnicas como PCA si hay muchas características correlacionadas, pero esto es opcional y debe justificarse en el EDA.

import numpy as np
# Feature Engineering
print("\nRealizando Ingeniería de Características...")

# TotalSF: Suma de TotalBsmtSF, 1stFlrSF y 2ndFlrSF
# Asegurarse de que las columnas existan y sean numéricas antes de sumar
sf_cols = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF']
# Verificar si todas las columnas necesarias existen y son numéricas
if all(col in df.columns and df[col].dtype in [np.number] for col in sf_cols):
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    print("Característica 'TotalSF' creada.")
else:
    print(f"No se pudo crear 'TotalSF'. Asegúrese de que las columnas {sf_cols} existan y sean numéricas.")


# Age: Diferencia entre YrSold y YearBuilt
# Asegurarse de que las columnas existan y sean numéricas antes de restar
age_cols = ['YrSold', 'YearBuilt']
if all(col in df.columns and df[col].dtype in [np.number] for col in age_cols):
    df['Age'] = df['YrSold'] - df['YearBuilt']
    # Considerar casas vendidas antes de ser construidas (posibles errores de datos)
    # Podríamos corregirlos o considerarlos como outliers. Aquí, simplemente los marcamos si existen.
    if (df['Age'] < 0).any():
        print("Advertencia: Se encontraron edades negativas en la característica 'Age'.")
    print("Característica 'Age' creada.")
else:
     print(f"No se pudo crear 'Age'. Asegúrese de que las columnas {age_cols} existan y sean numéricas.")


# Opcional: Crear otras características si son relevantes para el dominio
# Ejemplo: Ratio entre áreas (ej. GrLivArea/TotalSF), número total de baños, etc.
# df['Bathrooms'] = df['FullBath'] + 0.5 * df['HalfBath'] + df['BsmtFullBath'] + 0.5 * df['BsmtHalfBath']


print("\nIngeniería de Características completada.")

# Mostrar información actualizada del DataFrame para ver las nuevas columnas
print("\nInformación del DataFrame después de la Ingeniería de Características:")
df.info()

# Mostrar las primeras filas del DataFrame para ver los resultados
print("\nPrimeras filas del DataFrame después de la Ingeniería de Características:")
print(df.head())

# Análisis de Correlaciones para identificar características redundantes
print("\nAnalizando correlaciones entre características...")

# Calcular la matriz de correlación. Excluir columnas no numéricas si aún existen.
# Si ya todas las columnas relevantes son numéricas después de la codificación y escalado, usa todo el df.
numeric_df = df.select_dtypes(include=np.number)
correlation_matrix = numeric_df.corr()

# Visualizar la matriz de correlación (opcional, requiere matplotlib/seaborn)
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.figure(figsize=(12, 10))
# sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False) # annot=True si quieres ver los valores
# plt.title('Matriz de Correlación')
# plt.show()

# Identificar pares de características altamente correlacionadas
# Define un umbral de correlación (por ejemplo, 0.8 o 0.9)
correlation_threshold = 0.8

# Crear una máscara para la parte superior del triángulo de la matriz de correlación
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Encontrar las columnas con correlación absoluta alta
to_drop_high_corr = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]

print(f"\nColumnas con alta correlación (> {correlation_threshold}) detectadas:")
print(to_drop_high_corr)

# Decidir qué columna eliminar de los pares altamente correlacionados
# Esto requiere conocimiento del dominio o una justificación (ej. quedarse con la que tiene mayor correlación con la variable target)
# Para este ejemplo, simplemente mostraremos los pares y el usuario debe decidir cuál eliminar.

high_corr_pairs = []
for col1 in upper.columns:
    for col2 in upper.index:
        if col1 != col2 and abs(upper.loc[col2, col1]) > correlation_threshold:
             high_corr_pairs.append((col2, col1, upper.loc[col2, col1]))

print("\nPares de características altamente correlacionadas y su valor:")
for pair in high_corr_pairs:
    print(f"{pair[0]} y {pair[1]}: {pair[2]:.2f}")

# Ejemplo de decisión (MANUAL): Si GarageArea y GarageCars están altamente correlacionadas,
# podríamos decidir quedarnos con 'GarageCars' ya que representa el número de coches,
# lo cual podría ser más directamente relacionado con el valor percibido.

# Lista de columnas a considerar eliminar basadas en alta correlación (EJEMPLO - AJUSTAR SEGÚN ANÁLISIS)
# Por ejemplo, si GarageArea y GarageCars son muy correlacionadas, podríamos añadir 'GarageArea' aquí.
# Si TotalBsmtSF, 1stFlrSF, GrLivArea son muy correlacionadas con TotalSF, podríamos considerar eliminar algunas de las originales.
# Este paso es CRUCIAL y debe basarse en el análisis de los pares identificados.

# Ejemplo: Si 'GarageArea' y 'GarageCars' tienen alta correlación y decides eliminar 'GarageArea'
# columns_to_potentially_drop = ['GarageArea'] # Inicializa la lista de columnas a considerar eliminar

# Puedes iterar sobre los pares de alta correlación y aplicar una regla o decidir manualmente
# for col1, col2, corr_value in high_corr_pairs:
    # Decide cuál eliminar basándote en alguna métrica o dominio
    # if col1 == 'GarageArea' and col2 == 'GarageCars':
    #     columns_to_potentially_drop.append('GarageArea')
    # elif col1 == 'GrLivArea' and col2 == 'TotalSF':
    #      columns_to_potentially_drop.append('GrLivArea') # Ejemplo: quedarse con TotalSF si es más informativo

# Eliminar duplicados de la lista (una columna puede estar en múltiples pares)
# columns_to_potentially_drop = list(set(columns_to_potentially_drop))

# print(f"\nColumnas que se considerarán eliminar por alta correlación: {columns_to_potentially_drop}")

# if columns_to_potentially_drop:
#     # Asegurarse de que las columnas existen antes de intentar eliminarlas
#     existing_cols_to_drop = [col for col in columns_to_potentially_drop if col in df.columns]
#     if existing_cols_to_drop:
#         print(f"\nEliminando columnas altamente correlacionadas: {existing_cols_to_drop}")
#         df = df.drop(columns=existing_cols_to_drop)
#         print("Columnas eliminadas.")
#     else:
#          print("\nLas columnas a eliminar consideradas no existen en el DataFrame actual.")
# else:
#     print("\nNo hay columnas identificadas para eliminar por alta correlación (basado en el umbral y reglas de ejemplo).")

# Nota sobre PCA: PCA puede ser útil si hay muchas características numéricas correlacionadas y quieres reducir la dimensionalidad
# manteniendo la mayor parte de la varianza. Sin embargo, los componentes de PCA son combinaciones lineales
# de las características originales y son menos interpretables. Si el objetivo es la interpretabilidad,
# la eliminación de columnas altamente correlacionadas es a menudo preferible.

# PCA es una transformación, no una eliminación directa de columnas. Si se usa PCA,
# se reemplazarían las columnas numéricas correlacionadas por menos componentes principales.
# No se implementa PCA aquí, ya que es opcional y requiere justificación en el EDA.

# Mostrar información final del DataFrame
print("\nInformación final del DataFrame después de la ingeniería de características y consideración de correlaciones:")
df.info()

# Mostrar las primeras filas del DataFrame final
print("\nPrimeras filas del DataFrame final:")
print(df.head())

In [ ]:
# prompt: Análisis Exploratorio de Datos (EDA)
# Realiza visualizaciones clave:
# Histogramas y boxplots para SalePrice y variables numéricas para identificar distribución y outliers.
# Gráficos de barras para variables categóricas importantes (e.g., Neighborhood, OverallQual).
# Matriz de correlación (e.g., con seaborn.heatmap) para identificar relaciones entre características y SalePrice.
# Discute hallazgos, como:
# Si SalePrice tiene sesgo, considera transformaciones logarítmicas.
# Identifica outliers (e.g., casas con precios extremadamente altos o bajos) y decide si eliminarlos o manejarlos.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Visualizaciones clave

# Histograma y Boxplot para SalePrice
print("\nVisualizando la distribución de SalePrice...")
plt.figure(figsize=(12, 5))

# Histograma de SalePrice
plt.subplot(1, 2, 1)
sns.histplot(df['SalePrice'], kde=True)
plt.title('Distribución de SalePrice')
plt.xlabel('SalePrice')
plt.ylabel('Frecuencia')

# Boxplot de SalePrice
plt.subplot(1, 2, 2)
sns.boxplot(y=df['SalePrice'])
plt.title('Boxplot de SalePrice')
plt.ylabel('SalePrice')

plt.tight_layout()
plt.show()

# Discusión sobre SalePrice: Sesgo y Outliers
print("\nAnálisis de SalePrice:")
skewness = df['SalePrice'].skew()
print(f"Sesgo de SalePrice: {skewness:.2f}")

if abs(skewness) > 0.5: # Un umbral común para considerar un sesgo significativo
    print("SalePrice parece tener un sesgo significativo.")
    print("Considerar una transformación logarítmica (ej. np.log1p) podría ser beneficioso para la modelización.")

# Identificar outliers en SalePrice usando IQR
Q1 = df['SalePrice'].quantile(0.25)
Q3 = df['SalePrice'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['SalePrice'] < lower_bound) | (df['SalePrice'] > upper_bound)]
print(f"\nSe encontraron {len(outliers)} outliers en SalePrice (usando el método IQR).")
print("Decisión sobre manejar o eliminar outliers debe basarse en el contexto y el modelo a usar.")

# Visualizaciones para variables numéricas (ejemplo: algunas de las importantes)
numeric_cols_viz = ['GrLivArea', 'TotalBsmtSF', 'Age', 'TotalSF'] # Agrega otras columnas numéricas relevantes
print(f"\nVisualizando distribuciones y outliers para algunas variables numéricas: {numeric_cols_viz}")
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols_viz):
    if col in df.columns and df[col].dtype in [np.number]:
        plt.subplot(2, len(numeric_cols_viz), i + 1)
        sns.histplot(df[col], kde=True)
        plt.title(f'Distribución de {col}')
        plt.xlabel(col)
        plt.ylabel('Frecuencia')

        plt.subplot(2, len(numeric_cols_viz), i + len(numeric_cols_viz) + 1)
        sns.boxplot(y=df[col])
        plt.title(f'Boxplot de {col}')
        plt.ylabel(col)
    else:
        print(f"Advertencia: La columna '{col}' no existe o no es numérica para visualización.")

plt.tight_layout()
plt.show()

# Visualizaciones para variables categóricas importantes
# Selecciona algunas variables categóricas interesantes para visualizar
# Usa las columnas nominales y ordinales después de la imputación pero antes de la codificación
# (ya que pd.get_dummies convierte las nominales en múltiples columnas)
# Si ya ejecutaste la codificación, usa las columnas originales para esta visualización.
# Asumiendo que las columnas originales están disponibles o usando las columnas ordinales codificadas y algunas nominales clave originales si las tienes
# Si solo tienes el df codificado, puedes visualizar las ordinales (ahora numéricas) con boxplots o analizar las dummies.

# Si quieres visualizar las columnas nominales originales, necesitas el df antes de get_dummies.
# Para este ejemplo, asumiremos que podemos usar las columnas ordinales codificadas (ahora numéricas)
# y tal vez algunas de las columnas originales si no se han perdido.
# Si el código anterior ya corrió y transformó las columnas nominales, no podemos visualizarlas como barras fácilmente aquí.

# Vamos a visualizar las columnas ordinales codificadas (que ahora son numéricas)
ordinal_cols_viz = ordinal_cols # Usamos la lista de columnas ordinales definida anteriormente
print(f"\nVisualizando la relación entre SalePrice y columnas ordinales: {ordinal_cols_viz}")
plt.figure(figsize=(15, 8))
for i, col in enumerate(ordinal_cols_viz):
    if col in df.columns and df[col].dtype in [np.number] and 'SalePrice' in df.columns:
        plt.subplot(2, int(np.ceil(len(ordinal_cols_viz)/2)), i + 1)
        # Boxplot para ver la relación entre la categoría ordinal y SalePrice
        sns.boxplot(x=df[col], y=df['SalePrice'])
        plt.title(f'SalePrice vs {col}')
        plt.xlabel(col)
        plt.ylabel('SalePrice')
    else:
         print(f"Advertencia: La columna '{col}' o 'SalePrice' no existe o no es numérica para visualización ordinal.")

plt.tight_layout()
plt.show()


# Visualizar la relación entre SalePrice y algunas columnas nominales CLAVE (si las tienes disponibles en un formato adecuado)
# Si ya aplicaste One-Hot Encoding, la mejor manera es analizar la relación de SalePrice con los grupos de cada columna nominal original.
# Ejemplo usando 'Neighborhood' (si la columna original está disponible o si trabajas con el df antes de one-hot)
# Si 'Neighborhood' fue one-hoteada, necesitarías agrupar por el valor original.
# Asumiendo que puedes acceder al dataframe antes de la codificación o tienes la columna original
# Si no, este paso de visualización de categóricas con barras después de OHE es complicado.

# **Alternativa (si ya aplicaste OHE):** Puedes analizar la relación entre SalePrice y los grupos de una variable categórica original
# calculando la mediana de SalePrice por grupo.
# Si tienes el dataframe original o una copia antes de la codificación:
# df_original = pd.read_csv('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/AmesHousing.csv') # Cargar de nuevo o usar una copia
# if 'Neighborhood' in df_original.columns and 'SalePrice' in df_original.columns:
#     plt.figure(figsize=(15, 6))
#     sns.boxplot(x='Neighborhood', y='SalePrice', data=df_original)
#     plt.title('SalePrice por Neighborhood (original)')
#     plt.xticks(rotation=90)
#     plt.show()

# Si no tienes el df original disponible fácilmente aquí, puedes saltarte la visualización de barras para nominales después de OHE.

# Matriz de correlación con SalePrice
print("\nGenerando matriz de correlación con SalePrice...")

# Calcular la matriz de correlación solo para columnas numéricas
numeric_df_for_corr = df.select_dtypes(include=np.number)

# Calcular las correlaciones con SalePrice
# Asegurarse de que SalePrice está en el DataFrame numérico
if 'SalePrice' in numeric_df_for_corr.columns:
    correlation_with_saleprice = numeric_df_for_corr.corr()['SalePrice'].sort_values(ascending=False)
    print("\nCorrelación de variables numéricas con SalePrice:")
    print(correlation_with_saleprice)

    # Visualizar la matriz de correlación completa (puede ser grande)
    # plt.figure(figsize=(15, 12))
    # sns.heatmap(numeric_df_for_corr.corr(), cmap='coolwarm', annot=False) # annot=True puede ser muy denso
    # plt.title('Matriz de Correlación de Variables Numéricas')
    # plt.show()

    # Visualizar solo las correlaciones con SalePrice (más legible)
    plt.figure(figsize=(8, 10))
    sns.heatmap(correlation_with_saleprice.to_frame(), cmap='coolwarm', annot=True, fmt=".2f", cbar=False)
    plt.title('Correlación de Variables Numéricas con SalePrice')
    plt.yticks(rotation=0)
    plt.show()

else:
    print("\n'SalePrice' no se encontró en las columnas numéricas para calcular la correlación.")


# Discusión de hallazgos de correlación:
print("\nDiscusión de hallazgos de correlación:")
print("Las variables con los valores de correlación más altos (positivos o negativos) son las que tienen una relación lineal más fuerte con SalePrice.")
print("Ejemplos de variables con alta correlación positiva suelen ser 'OverallQual', 'GrLivArea', 'TotalBsmtSF', 'GarageCars'.")
print("Ejemplos de variables con alta correlación negativa pueden ser menos comunes, pero podrían existir.")
print("Considerar estas variables más correlacionadas como potencialmente importantes para el modelo.")

# Resumen general del EDA y próximos pasos
print("\nResumen del Análisis Exploratorio de Datos (EDA):")
print("- Se analizó la distribución de SalePrice, identificando un posible sesgo y outliers.")
print("- Se visualizaron distribuciones y outliers para variables numéricas clave.")
print("- Se analizó la relación entre SalePrice y variables ordinales.")
# Si pudiste visualizar nominales: print("- Se visualizaron las distribuciones de variables categóricas importantes y su relación con SalePrice.")
print("- Se calculó y visualizó la matriz de correlación, identificando variables fuertemente relacionadas con SalePrice.")
# Si analizaste alta correlación entre features: print("- Se identificaron pares de características altamente correlacionadas.")

print("\nPróximos pasos:")
print("- Decidir el tratamiento para el sesgo de SalePrice (ej. transformación logarítmica).")
print("- Decidir cómo manejar los outliers (eliminar, transformar, usar modelos robustos).")
print("- Refinar la selección de características basadas en el análisis de correlación y la importancia percibida.")
print("- Preparar los datos finales para el entrenamiento del modelo (separación en conjuntos de entrenamiento/prueba).")
print("- Seleccionar y entrenar modelos de aprendizaje automático para predecir SalePrice.")
print("- Evaluar el rendimiento de los modelos.")

In [ ]:
# prompt: Genera estadísticas descriptivas (media, mediana, desviación estándar, etc.) para las variables numéricas.
# Crea histogramas para variables numéricas importantes (por ejemplo, 'SalePrice', 'GrLivArea', 'TotalBsmtSF').
# Crea gráficas de barras para variables categóricas relevantes (por ejemplo, 'Neighborhood', 'OverallQual').
# Genera una matriz de correlación y su correspondiente heatmap para las variables numéricas.
# Identifica y visualiza cualquier patrón o tendencia en los datos.

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Generar estadísticas descriptivas para las variables numéricas
print("\nEstadísticas descriptivas para variables numéricas:")
# Selecciona solo las columnas numéricas para las estadísticas descriptivas
df_numeric = df.select_dtypes(include=np.number)
print(df_numeric.describe())


# Crear histogramas para variables numéricas importantes
# Ya hemos generado histogramas en la sección anterior, pero podemos volver a ejecutarlos o seleccionar otras variables si es necesario.
# Las variables 'SalePrice', 'GrLivArea', 'TotalBsmtSF' ya fueron visualizadas.
# Podemos añadir '1stFlrSF', 'GarageArea', 'YearBuilt' como ejemplos adicionales.
numeric_cols_hist = ['1stFlrSF', 'GarageArea', 'YearBuilt']
print(f"\nCreando histogramas para: {numeric_cols_hist}")
plt.figure(figsize=(15, 5))
for i, col in enumerate(numeric_cols_hist):
    if col in df.columns and df[col].dtype in [np.number]:
        plt.subplot(1, len(numeric_cols_hist), i + 1)
        sns.histplot(df[col], kde=True)
        plt.title(f'Distribución de {col}')
        plt.xlabel(col)
        plt.ylabel('Frecuencia')
    else:
        print(f"Advertencia: La columna '{col}' no existe o no es numérica para histograma.")
plt.tight_layout()
plt.show()


# Crear gráficas de barras para variables categóricas relevantes
# Para graficar variables categóricas relevantes, necesitamos las columnas antes de One-Hot Encoding.
# Si el código anterior ya ejecutó One-Hot Encoding, las columnas nominales ya no existen en su formato original.
# Podemos volver a cargar el dataframe original o usar una copia antes de la codificación.
# Asumiendo que trabajamos con el dataframe original o una copia para esta visualización.
# NOTA: Si solo tienes el dataframe después de OHE, puedes omitir este paso o encontrar una alternativa de visualización.

# Vamos a asumir que podemos usar el dataframe original para visualizar las columnas categóricas.
# Cargar el dataframe original si no está disponible en su estado pre-codificado.
try:
    df_original = pd.read_csv('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/AmesHousing.csv')
    print("\nCargado el DataFrame original para visualizaciones categóricas.")

    # Seleccionar columnas categóricas relevantes para gráfica de barras
    # Usar columnas que NO fueron imputadas con 'missing' para barras simples, a menos que 'missing' sea una categoría significativa.
    # Columnas como 'Neighborhood', 'OverallQual' (si se trata como categórica), 'MSZoning'
    # 'OverallQual' es numérica, usaremos otras categóricas como 'Neighborhood' y 'MSZoning'.
    categorical_cols_bar = ['Neighborhood', 'MSZoning'] # Agrega otras columnas categóricas relevantes

    print(f"\nCreando gráficas de barras para: {categorical_cols_bar}")
    plt.figure(figsize=(15, 6))
    for i, col in enumerate(categorical_cols_bar):
        if col in df_original.columns and df_original[col].dtype == 'object':
            plt.subplot(1, len(categorical_cols_bar), i + 1)
            # Contar la frecuencia de cada categoría
            sns.countplot(y=col, data=df_original, order=df_original[col].value_counts().index)
            plt.title(f'Distribución de {col}')
            plt.xlabel('Frecuencia')
            plt.ylabel(col)
        else:
            print(f"Advertencia: La columna '{col}' no existe o no es categórica en el DataFrame original para gráfica de barras.")
    plt.tight_layout()
    plt.show()

except FileNotFoundError:
    print("\nNo se pudo cargar el DataFrame original. Saltando la creación de gráficas de barras para columnas categóricas.")
    print("Asegúrese de que el archivo 'AmesHousing.csv' esté disponible en la ruta especificada si desea estas visualizaciones.")


# Generar una matriz de correlación y su correspondiente heatmap para las variables numéricas
# Ya hemos calculado y visualizado la matriz de correlación numérico_df_for_corr.corr()
# Podemos visualizar el heatmap completo aquí si es necesario, aunque puede ser denso.
# La visualización de correlación con SalePrice (univariada) ya se hizo.

print("\nGenerando heatmap de la matriz de correlación para variables numéricas...")
# Calcular la matriz de correlación solo para columnas numéricas del dataframe actual (post-procesamiento)
df_numeric_current = df.select_dtypes(include=np.number)
correlation_matrix_current = df_numeric_current.corr()

plt.figure(figsize=(15, 12))
# Usar annot=False para evitar que sea ilegible en datasets grandes, o ajustar el tamaño de la figura.
sns.heatmap(correlation_matrix_current, cmap='coolwarm', annot=False)
plt.title('Heatmap de la Matriz de Correlación (Variables Numéricas Post-Procesamiento)')
plt.show()


# Identificar y visualizar cualquier patrón o tendencia en los datos
# Esto es una tarea amplia, pero podemos visualizar relaciones clave usando scatter plots.
# Visualizar la relación entre 'SalePrice' y las variables numéricas más correlacionadas identificadas anteriormente.

print("\nVisualizando patrones y tendencias con scatter plots...")

# Seleccionar las N variables más correlacionadas con SalePrice (excluyendo SalePrice misma)
if 'SalePrice' in df_numeric_current.columns:
    correlation_with_saleprice = df_numeric_current.corr()['SalePrice'].sort_values(ascending=False)
    # Seleccionar las top N variables (excluyendo SalePrice)
    top_n = 5 # Número de variables para visualizar
    top_correlated_cols = correlation_with_saleprice.head(top_n + 1).index.tolist()
    if 'SalePrice' in top_correlated_cols:
        top_correlated_cols.remove('SalePrice')

    print(f"\nVisualizando la relación de SalePrice con las {top_n} variables más correlacionadas: {top_correlated_cols}")
    plt.figure(figsize=(15, 5 * int(np.ceil(len(top_correlated_cols)/3)))) # Ajustar el tamaño de la figura
    for i, col in enumerate(top_correlated_cols):
         if col in df_numeric_current.columns:
            plt.subplot(int(np.ceil(len(top_correlated_cols)/3)), 3, i + 1)
            sns.scatterplot(x=df_numeric_current[col], y=df_numeric_current['SalePrice'])
            plt.title(f'SalePrice vs {col}')
            plt.xlabel(col)
            plt.ylabel('SalePrice')
         else:
             print(f"Advertencia: La columna '{col}' no está disponible para scatter plot.")
    plt.tight_layout()
    plt.show()

else:
    print("\n'SalePrice' no está disponible para crear scatter plots de correlación.")


# Otros patrones a considerar (requieren análisis manual):
# - Relaciones no lineales (observar scatter plots)
# - Interacciones entre características (requiere crear nuevas características o probar modelos con interacciones)
# - Patrones temporales si existen (ej. 'YearBuilt', 'YrSold') - 'Age' ya ayuda aquí.

print("\nAnálisis de patrones y tendencias adicionales (requiere inspección visual):")
print("- Observa los scatter plots para identificar relaciones lineales o no lineales.")
print("- Considera cómo 'Age' se relaciona con 'SalePrice'.")
print("- Revisa las distribuciones de variables categóricas importantes para entender la composición del dataset.")
print("- La matriz de correlación (heatmap) ayuda a ver grupos de variables relacionadas.")


In [ ]:
# prompt: Usa train_test_split de sklearn para dividir en entrenamiento (80%) y prueba (20%).
# Dado que el examen requiere validación cruzada de 5 folds, usa KFold de sklearn para asegurar consistencia en la evaluación.
# Nota: Como es un problema de regresión, KFold es adecuado; no necesitas StratifiedKFold, ya que no hay clases balanceadas.

import numpy as np
from sklearn.model_selection import train_test_split, KFold

# Separar las características (X) y la variable objetivo (y)
# Asumimos que 'SalePrice' es la variable objetivo
if 'SalePrice' in df.columns:
    X = df.drop('SalePrice', axis=1)
    y = df['SalePrice']
    print("\nCaracterísticas (X) y variable objetivo (y) separadas.")
else:
    print("\nError: 'SalePrice' no se encontró en el DataFrame. No se puede separar X e y.")

# Asegurarse de que todas las columnas en X son numéricas antes de la división
X = X.select_dtypes(include=np.number)
# Eliminar cualquier fila con NaN que pudiera haber quedado (aunque con la imputación no debería haber)
X = X.dropna()
y = y.loc[X.index] # Asegurar que y coincide con las filas de X

print(f"\nForma de X antes de la división: {X.shape}")
print(f"Forma de y antes de la división: {y.shape}")


# División en conjuntos de entrenamiento y prueba
# Stratify no se usa porque es un problema de regresión, no de clasificación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # random_state para reproducibilidad

print(f"\nForma del conjunto de entrenamiento (X_train): {X_train.shape}")
print(f"Forma del conjunto de prueba (X_test): {X_test.shape}")
print(f"Forma de la variable objetivo de entrenamiento (y_train): {y_train.shape}")
print(f"Forma de la variable objetivo de prueba (y_test): {y_test.shape}")

# Configurar KFold para la validación cruzada
n_splits = 5 # Número de folds para la validación cruzada
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42) # shuffle=True y random_state para mezclar los datos consistentemente

print(f"\nConfiguración de KFold con {n_splits} folds:")
print(kf)

# Ejemplo de cómo usar KFold (esto se usa DENTRO del proceso de entrenamiento del modelo)
# El siguiente bucle es solo para demostrar cómo iterar sobre los folds.
# En la práctica, pasarías el objeto kf a una función como cross_val_score o GridSearchCV.

print("\nGenerando índices para los folds de validación cruzada (ejemplo):")
fold_indices = []
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    print(f"Fold {fold + 1}:")
    print(f"  Índices de entrenamiento: {train_index[:10]}... ({len(train_index)} total)") # Mostrar solo los primeros 10
    print(f"  Índices de validación:   {val_index[:10]}... ({len(val_index)} total)")    # Mostrar solo los primeros 10
    fold_indices.append((train_index, val_index))

print("\nValidación cruzada configurada.")


# LinearRegression

In [ ]:
# prompt: Implementa LinearRegression en Python con sklearn, usando validación cruzada de 5 folds, y reporta MAE, MSE, R2 y MAPE.

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
import numpy as np

# Definir las métricas de evaluación
# MAE: Error Absoluto Medio
# MSE: Error Cuadrático Medio
# R2: Coeficiente de Determinación
# MAPE: Error Porcentual Absoluto Medio

def mean_absolute_percentage_error(y_true, y_pred):
    """Calcula el Error Porcentual Absoluto Medio (MAPE)."""
    # Evitar división por cero
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Reemplazar ceros en y_true para evitar inf o nan.
    # Se puede usar un pequeño épsilon o filtrar. Aquí usamos un épsilon.
    epsilon = 1e-8
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# Crear objetos scorer para las métricas (MAPE necesita un scorer personalizado)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False) # greater_is_better=False para métricas de error
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score, greater_is_better=True)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# Inicializar el modelo de Regresión Lineal
model = LinearRegression()

# Realizar validación cruzada con 5 folds
# cross_val_score por defecto usa la métrica 'score' del estimador (que para LinearRegression es R2)
# Podemos pasar nuestros propios scorers para evaluar múltiples métricas
print("\nRealizando validación cruzada con Linear Regression...")

# Usamos cross_validate para obtener múltiples métricas simultáneamente
from sklearn.model_selection import cross_validate

scoring = {
    'mae': mae_scorer,
    'mse': mse_scorer,
    'r2': r2_scorer,
    'mape': mape_scorer
}

cv_results = cross_validate(model, X_train, y_train, cv=kf, scoring=scoring)

# Los resultados de cross_validate son diccionarios. Accedemos a los arrays de resultados.
# Las métricas de error (MAE, MSE, MAPE) vienen negativas porque make_scorer las optimiza para maximizar (por defecto).
# Necesitamos tomar el valor absoluto para el reporte.
cv_mae = -cv_results['test_mae']
cv_mse = -cv_results['test_mse']
cv_r2 = cv_results['test_r2']
cv_mape = -cv_results['test_mape']

# Reportar los resultados promedio y desviación estándar de cada métrica
print(f"\nResultados de Validación Cruzada (promedio +/- std dev):")
print(f"  MAE:  {cv_mae.mean():.4f} +/- {cv_mae.std():.4f}")
print(f"  MSE:  {cv_mse.mean():.4f} +/- {cv_mse.std():.4f}")
print(f"  R2:   {cv_r2.mean():.4f} +/- {cv_r2.std():.4f}")
print(f"  MAPE: {cv_mape.mean():.4f}% +/- {cv_mape.std():.4f}%")

# Opcional: Entrenar el modelo final en todo el conjunto de entrenamiento y evaluar en el conjunto de prueba
# Este paso es típicamente posterior a la selección del modelo y hyperparámetros
# print("\nEntrenando modelo final en el conjunto de entrenamiento completo...")
# model.fit(X_train, y_train)
# print("Evaluando en el conjunto de prueba...")
# y_pred_test = model.predict(X_test)

# # Calcular métricas en el conjunto de prueba
# test_mae = mean_absolute_error(y_test, y_pred_test)
# test_mse = mean_squared_error(y_test, y_pred_test)
# test_r2 = r2_score(y_test, y_pred_test)
# test_mape = mean_absolute_percentage_error(y_test, y_pred_test)


# print("\nMétricas en el conjunto de prueba:")
# print(f"  MAE:  {test_mae:.4f}")
# print(f"  MSE:  {test_mse:.4f}")
# print(f"  R2:   {test_r2:.4f}")
# print(f"  MAPE: {test_mape:.4f}%")



# Lasso

In [ ]:
# prompt: Implementa Lasso en Python con sklearn, optimiza alpha usando GridSearchCV, RandomizedSearchCV y BayesSearchCV de skopt, con rangos logarítmicos, y reporta MAE, MSE, R2 y MAPE con validación cruzada de 5 folds."
# Rango: alpha = [0.001, 0.01, 0.1, 1, 10] (Grid), loguniform(0.001, 10) (Random/Bayesian).
# Justificación: Alpha controla la regularización L1, y rangos logarítmicos cubren fuerzas variadas, útil para selección de características.

!pip install scikit-optimize
!pip install scipy

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import time
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import cross_validate
from skopt import BayesSearchCV
from skopt.space import Real
from scipy.stats import loguniform # Import loguniform from scipy

# Ensure scorers are defined (as per the original code context)
def mean_absolute_percentage_error(y_true, y_pred):
    """Calcula el Error Porcentual Absoluto Medio (MAPE)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    epsilon = 1e-8
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score, greater_is_better=True)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

scoring = {
    'mae': mae_scorer,
    'mse': mse_scorer,
    'r2': r2_scorer,
    'mape': mape_scorer
}

scoring_optimizer = {'mae': mae_scorer}


# Definir el modelo Lasso
lasso = Lasso(random_state=42, max_iter=10000) # Aumentar max_iter si no converge


# --- Optimización con GridSearchCV ---
print("\nIniciando optimización con GridSearchCV para Lasso...")

# Rango de alpha para GridSearchCV (escala logarítmica)
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]} # Rango específico para Grid

grid_search = GridSearchCV(estimator=lasso, param_grid=param_grid,
                           scoring=scoring_optimizer, refit='mae', # Optimizar usando MAE
                           cv=kf, verbose=1, n_jobs=-1)

start_time_grid = time.time()
grid_search.fit(X_train, y_train)
end_time_grid = time.time()

print("\nResultados de GridSearchCV:")
print(f"Mejores hiperparámetros encontrados: {grid_search.best_params_}")
print(f"Mejor MAE promedio en validación: {-grid_search.best_score_:.4f}")
print(f"Tiempo de ejecución de GridSearchCV: {end_time_grid - start_time_grid:.2f} segundos")

best_lasso_grid = grid_search.best_estimator_

print("\nEvaluando el mejor modelo de GridSearchCV con validación cruzada completa:")
cv_results_grid_best = cross_validate(best_lasso_grid, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_grid = -cv_results_grid_best['test_mae']
cv_mse_grid = -cv_results_grid_best['test_mse']
cv_r2_grid = cv_results_grid_best['test_r2']
cv_mape_grid = -cv_results_grid_best['test_mape']

print(f"  MAE:  {cv_mae_grid.mean():.4f} +/- {cv_mae_grid.std():.4f}")
print(f"  MSE:  {cv_mse_grid.mean():.4f} +/- {cv_mse_grid.std():.4f}")
print(f"  R2:   {cv_r2_grid.mean():.4f} +/- {cv_r2_grid.std():.4f}")
print(f"  MAPE: {cv_mape_grid.mean():.4f}% +/- {cv_mape_grid.std():.4f}%")


# --- Optimización con RandomizedSearchCV ---
print("\nIniciando optimización con RandomizedSearchCV para Lasso...")

# Rango de alpha para RandomizedSearchCV (distribución loguniform usando scipy)
param_dist = {'alpha': loguniform(0.001, 10)} # Use loguniform from scipy

# Número de iteraciones (puntos a probar). Ajusta este valor según el tiempo disponible.
n_iter_rand = 50 # Ejemplo: probar 50 combinaciones aleatorias

random_search = RandomizedSearchCV(estimator=lasso, param_distributions=param_dist,
                                   n_iter=n_iter_rand,
                                   scoring=scoring_optimizer, refit='mae', # Optimizar usando MAE
                                   cv=kf, verbose=1, random_state=42, n_jobs=-1) # random_state para reproducibilidad

start_time_rand = time.time()
random_search.fit(X_train, y_train)
end_time_rand = time.time()

print("\nResultados de RandomizedSearchCV:")
print(f"Mejores hiperparámetros encontrados: {random_search.best_params_}")
print(f"Mejor MAE promedio en validación: {-random_search.best_score_:.4f}")
print(f"Tiempo de ejecución de RandomizedSearchCV: {end_time_rand - start_time_rand:.2f} segundos")

best_lasso_rand = random_search.best_estimator_

print("\nEvaluando el mejor modelo de RandomizedSearchCV con validación cruzada completa:")
cv_results_rand_best = cross_validate(best_lasso_rand, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_rand = -cv_results_rand_best['test_mae']
cv_mse_rand = -cv_results_rand_best['test_mse']
cv_r2_rand = cv_results_rand_best['test_r2']
cv_mape_rand = -cv_results_rand_best['test_mape']

print(f"  MAE:  {cv_mae_rand.mean():.4f} +/- {cv_mae_rand.std():.4f}")
print(f"  MSE:  {cv_mse_rand.mean():.4f} +/- {cv_mse_rand.std():.4f}")
print(f"  R2:   {cv_r2_rand.mean():.4f} +/- {cv_r2_rand.std():.4f}")
print(f"  MAPE: {cv_mape_rand.mean():.4f}% +/- {cv_mape_rand.std():.4f}%")


# --- Optimización con BayesSearchCV ---
print("\nIniciando optimización con BayesSearchCV para Lasso...")

# Rango de alpha para BayesSearchCV (espacio de búsqueda de skopt)
search_spaces = {'alpha': Real(0.001, 10, prior='log-uniform')}

# Número de iteraciones (puntos a explorar). Generalmente requiere menos que RandomizedSearch.
n_iter_bayes = 50 # Ejemplo: probar 50 iteraciones


# Definir BayesSearchCV
bayes_search = BayesSearchCV(estimator=lasso, search_spaces=search_spaces,
                             n_iter=n_iter_bayes,
                             scoring=scoring_optimizer, refit='mae', # Optimizar usando MAE
                             cv=kf, verbose=1, random_state=42, n_jobs=-1)

start_time_bayes = time.time()
bayes_search.fit(X_train, y_train)
end_time_bayes = time.time()

print("\nResultados de BayesSearchCV:")
print(f"Mejores hiperparámetros encontrados: {bayes_search.best_params_}")
print(f"Mejor MAE promedio en validación: {-bayes_search.best_score_:.4f}")
print(f"Tiempo de ejecución de BayesSearchCV: {end_time_bayes - start_time_bayes:.2f} segundos")

best_lasso_bayes = bayes_search.best_estimator_

print("\nEvaluando el mejor modelo de BayesSearchCV con validación cruzada completa:")
cv_results_bayes_best = cross_validate(best_lasso_bayes, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_bayes = -cv_results_bayes_best['test_mae']
cv_mse_bayes = -cv_results_bayes_best['test_mse']
cv_r2_bayes = cv_results_bayes_best['test_r2']
cv_mape_bayes = -cv_results_bayes_best['test_mape']

print(f"  MAE:  {cv_mae_bayes.mean():.4f} +/- {cv_mae_bayes.std():.4f}")
print(f"  MSE:  {cv_mse_bayes.mean():.4f} +/- {cv_mse_bayes.std():.4f}")
print(f"  R2:   {cv_r2_bayes.mean():.4f} +/- {cv_r2_bayes.std():.4f}")
print(f"  MAPE: {cv_mape_bayes.mean():.4f}% +/- {cv_mape_bayes.std():.4f}%")


# --- Comparación de resultados ---
print("\n--- Resumen de Resultados de Optimización (MAE promedio en validación) ---")
print(f"GridSearchCV:      {-grid_search.best_score_:.4f}")
print(f"RandomizedSearchCV: {-random_search.best_score_:.4f}")
print(f"BayesSearchCV:      {-bayes_search.best_score_:.4f}")

print("\n--- Reporte Completo de Métricas (Promedio de 5-fold CV) ---")

print("\nGridSearchCV Mejor Modelo:")
print(f"  MAE:  {cv_mae_grid.mean():.4f}")
print(f"  MSE:  {cv_mse_grid.mean():.4f}")
print(f"  R2:   {cv_r2_grid.mean():.4f}")
print(f"  MAPE: {cv_mape_grid.mean():.4f}%")

print("\nRandomizedSearchCV Mejor Modelo:")
print(f"  MAE:  {cv_mae_rand.mean():.4f}")
print(f"  MSE:  {cv_mse_rand.mean():.4f}")
print(f"  R2:   {cv_r2_rand.mean():.4f}")
print(f"  MAPE: {cv_mape_rand.mean():.4f}%")

print("\nBayesSearchCV Mejor Modelo:")
print(f"  MAE:  {cv_mae_bayes.mean():.4f}")
print(f"  MSE:  {cv_mse_bayes.mean():.4f}")
print(f"  R2:   {cv_r2_bayes.mean():.4f}")
print(f"  MAPE: {cv_mape_bayes.mean():.4f}%")

# Puedes seleccionar el mejor modelo general basado en el MAE promedio de validación
# Por ejemplo, si BayesSearchCV dio el mejor resultado:
# final_best_model = best_lasso_bayes
# print("\nEl mejor modelo general basado en MAE promedio de validación es el de BayesSearchCV.")


# Opcional: Entrenar el modelo final elegido en todo X_train y evaluar en X_test
# final_best_model = best_lasso_bayes # O grid_search.best_estimator_ o random_search.best_estimator_

# print("\nEntrenando el modelo final seleccionado en el conjunto de entrenamiento completo...")
# final_best_model.fit(X_train, y_train)
# print("Evaluando en el conjunto de prueba (X_test)...")
# y_pred_test_final = final_best_model.predict(X_test)

# # Calcular métricas en el conjunto de prueba con el modelo final
# test_mae_final = mean_absolute_error(y_test, y_pred_test_final)
# test_mse_final = mean_squared_error(y_test, y_pred_test_final)
# test_r2_final = r2_score(y_test, y_pred_test_final)
# test_mape_final = mean_absolute_percentage_error(y_test, y_pred_test_final)

# print("\nMétricas en el conjunto de prueba con el modelo Lasso final seleccionado:")
# print(f"  MAE:  {test_mae_final:.4f}")
# print(f"  MSE:  {test_mse_final:.4f}")
# print(f"  R2:   {test_r2_final:.4f}")
# print(f"  MAPE: {test_mape_final:.4f}%")

# ElasticNet

In [ ]:
# prompt: Implementa ElasticNet en Python con sklearn, optimiza alpha y l1_ratio usando GridSearchCV, RandomizedSearchCV y BayesSearchCV de skopt, y reporta MAE, MSE, R2 y MAPE con validación cruzada de 5 folds."
# Rango: alpha = [0.001, 0.01, 0.1, 1], l1_ratio = [0.1, 0.3, 0.5] (Grid); alpha loguniform(0.001, 1), l1_ratio uniform(0, 1) (Random/Bayesian).
# Justificación: Alpha regula fuerza, l1_ratio balancea L1/L2; rangos logarítmicos para alpha capturan variabilidad, l1_ratio cubre todo espectro.

from scipy.stats import loguniform, uniform
from sklearn.linear_model import ElasticNet
!pip install scikit-optimize

import time
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real

# Ensure scorers are defined (as per the original code context)
def mean_absolute_percentage_error(y_true, y_pred):
    """Calcula el Error Porcentual Absoluto Medio (MAPE)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    epsilon = 1e-8
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score, greater_is_better=True)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

scoring = {
    'mae': mae_scorer,
    'mse': mse_scorer,
    'r2': r2_scorer,
    'mape': mape_scorer
}

scoring_optimizer = {'mae': mae_scorer}


# Definir el modelo ElasticNet
elastic_net = ElasticNet(random_state=42, max_iter=10000) # Aumentar max_iter si no converge

# Assuming X_train, y_train, and kf are defined in previous cells

# --- Optimization with GridSearchCV ---
print("\nIniciando optimización con GridSearchCV para ElasticNet...")

# Define parameter grid for GridSearchCV
# alpha = [0.001, 0.01, 0.1, 1], l1_ratio = [0.1, 0.3, 0.5]
param_grid_en = {
    'alpha': [0.001, 0.01, 0.1, 1],
    'l1_ratio': [0.1, 0.3, 0.5]
}

grid_search_en = GridSearchCV(estimator=elastic_net, param_grid=param_grid_en,
                              scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                              cv=kf, verbose=1, n_jobs=-1)

start_time_grid_en = time.time()
grid_search_en.fit(X_train, y_train)
end_time_grid_en = time.time()

print("\nResultados de GridSearchCV para ElasticNet:")
print(f"Mejores hiperparámetros encontrados: {grid_search_en.best_params_}")
print(f"Mejor MAE promedio en validación: {-grid_search_en.best_score_:.4f}")
print(f"Tiempo de ejecución de GridSearchCV: {end_time_grid_en - start_time_grid_en:.2f} segundos")

best_elastic_net_grid = grid_search_en.best_estimator_

print("\nEvaluando el mejor modelo de GridSearchCV para ElasticNet con validación cruzada completa:")
cv_results_grid_best_en = cross_validate(best_elastic_net_grid, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_grid_en = -cv_results_grid_best_en['test_mae']
cv_mse_grid_en = -cv_results_grid_best_en['test_mse']
cv_r2_grid_en = cv_results_grid_best_en['test_r2']
cv_mape_grid_en = -cv_results_grid_best_en['test_mape']

print(f"  MAE:  {cv_mae_grid_en.mean():.4f} +/- {cv_mae_grid_en.std():.4f}")
print(f"  MSE:  {cv_mse_grid_en.mean():.4f} +/- {cv_mse_grid_en.std():.4f}")
print(f"  R2:   {cv_r2_grid_en.mean():.4f} +/- {cv_r2_grid_en.std():.4f}")
print(f"  MAPE: {cv_mape_grid_en.mean():.4f}% +/- {cv_mape_grid_en.std():.4f}%")


# --- Optimization with RandomizedSearchCV ---
print("\nIniciando optimización con RandomizedSearchCV para ElasticNet...")

# Define parameter distribution for RandomizedSearchCV
# alpha loguniform(0.001, 1), l1_ratio uniform(0, 1)
param_dist_en = {
    'alpha': loguniform(0.001, 1),
    'l1_ratio': uniform(0, 1)  # Use scipy.stats.uniform for RandomizedSearchCV
}

# Number of iterations (adjust as needed)
n_iter_rand_en = 50

random_search_en = RandomizedSearchCV(estimator=elastic_net, param_distributions=param_dist_en,
                                      n_iter=n_iter_rand_en,
                                      scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                                      cv=kf, verbose=1, random_state=42, n_jobs=-1) # random_state para reproducibilidad

start_time_rand_en = time.time()
random_search_en.fit(X_train, y_train)
end_time_rand_en = time.time()

print("\nResultados de RandomizedSearchCV para ElasticNet:")
print(f"Mejores hiperparámetros encontrados: {random_search_en.best_params_}")
print(f"Mejor MAE promedio en validación: {-random_search_en.best_score_:.4f}")
print(f"Tiempo de ejecución de RandomizedSearchCV: {end_time_rand_en - start_time_rand_en:.2f} segundos")

best_elastic_net_rand = random_search_en.best_estimator_

print("\nEvaluando el mejor modelo de RandomizedSearchCV para ElasticNet con validación cruzada completa:")
cv_results_rand_best_en = cross_validate(best_elastic_net_rand, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_rand_en = -cv_results_rand_best_en['test_mae']
cv_mse_rand_en = -cv_results_rand_best_en['test_mse']
cv_r2_rand_en = cv_results_rand_best_en['test_r2']
cv_mape_rand_en = -cv_results_rand_best_en['test_mape']

print(f"  MAE:  {cv_mae_rand_en.mean():.4f} +/- {cv_mae_rand_en.std():.4f}")
print(f"  MSE:  {cv_mse_rand_en.mean():.4f} +/- {cv_mse_rand_en.std():.4f}")
print(f"  R2:   {cv_r2_rand_en.mean():.4f} +/- {cv_r2_rand_en.std():.4f}")
print(f"  MAPE: {cv_mape_rand_en.mean():.4f}% +/- {cv_mape_rand_en.std():.4f}%")


# --- Optimization with BayesSearchCV ---
print("\nIniciando optimización con BayesSearchCV para ElasticNet...")

# Define search spaces for BayesSearchCV
# alpha loguniform(0.001, 1), l1_ratio uniform(0, 1)
search_spaces_en = {
    'alpha': Real(0.001, 1, prior='log-uniform'),
    'l1_ratio': Real(0, 1, prior='uniform') # Use skopt's Real for BayesSearchCV
}

# Number of iterations (adjust as needed)
n_iter_bayes_en = 50

bayes_search_en = BayesSearchCV(estimator=elastic_net, search_spaces=search_spaces_en,
                                n_iter=n_iter_bayes_en,
                                scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                                cv=kf, verbose=1, random_state=42, n_jobs=-1)

start_time_bayes_en = time.time()
bayes_search_en.fit(X_train, y_train)
end_time_bayes_en = time.time()

print("\nResultados de BayesSearchCV para ElasticNet:")
print(f"Mejores hiperparámetros encontrados: {bayes_search_en.best_params_}")
print(f"Mejor MAE promedio en validación: {-bayes_search_en.best_score_:.4f}")
print(f"Tiempo de ejecución de BayesSearchCV: {end_time_bayes_en - start_time_bayes_en:.2f} segundos")

best_elastic_net_bayes = bayes_search_en.best_estimator_

print("\nEvaluando el mejor modelo de BayesSearchCV para ElasticNet con validación cruzada completa:")
cv_results_bayes_best_en = cross_validate(best_elastic_net_bayes, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_bayes_en = -cv_results_bayes_best_en['test_mae']
cv_mse_bayes_en = -cv_results_bayes_best_en['test_mse']
cv_r2_bayes_en = cv_results_bayes_best_en['test_r2']
cv_mape_bayes_en = -cv_results_bayes_best_en['test_mape']

print(f"  MAE:  {cv_mae_bayes_en.mean():.4f} +/- {cv_mae_bayes_en.std():.4f}")
print(f"  MSE:  {cv_mse_bayes_en.mean():.4f} +/- {cv_mse_bayes_en.std():.4f}")
print(f"  R2:   {cv_r2_bayes_en.mean():.4f} +/- {cv_r2_bayes_en.std():.4f}")
print(f"  MAPE: {cv_mape_bayes_en.mean():.4f}% +/- {cv_mape_bayes_en.std():.4f}%")


# --- Comparación de resultados de ElasticNet ---
print("\n--- Resumen de Resultados de Optimización para ElasticNet (MAE promedio en validación) ---")
print(f"GridSearchCV:      {-grid_search_en.best_score_:.4f}")
print(f"RandomizedSearchCV: {-random_search_en.best_score_:.4f}")
print(f"BayesSearchCV:      {-bayes_search_en.best_score_:.4f}")

print("\n--- Reporte Completo de Métricas para ElasticNet (Promedio de 5-fold CV) ---")

print("\nGridSearchCV Mejor Modelo (ElasticNet):")
print(f"  MAE:  {cv_mae_grid_en.mean():.4f}")
print(f"  MSE:  {cv_mse_grid_en.mean():.4f}")
print(f"  R2:   {cv_r2_grid_en.mean():.4f}")
print(f"  MAPE: {cv_mape_grid_en.mean():.4f}%")

print("\nRandomizedSearchCV Mejor Modelo (ElasticNet):")
print(f"  MAE:  {cv_mae_rand_en.mean():.4f}")
print(f"  MSE:  {cv_mse_rand_en.mean():.4f}")
print(f"  R2:   {cv_r2_rand_en.mean():.4f}")
print(f"  MAPE: {cv_mape_rand_en.mean():.4f}%")

print("\nBayesSearchCV Mejor Modelo (ElasticNet):")
print(f"  MAE:  {cv_mae_bayes_en.mean():.4f}")
print(f"  MSE:  {cv_mse_bayes_en.mean():.4f}")
print(f"  R2:   {cv_r2_bayes_en.mean():.4f}")
print(f"  MAPE: {cv_mape_bayes_en.mean():.4f}%")

# You can select the overall best ElasticNet model based on validation MAE
# For instance, if BayesSearchCV gave the best result:
# final_best_elastic_net_model = best_elastic_net_bayes
# print("\nEl mejor modelo ElasticNet general basado en MAE promedio de validación es el de BayesSearchCV.")

# Optional: Train the chosen best ElasticNet model on the entire X_train and evaluate on X_test
# final_best_elastic_net_model = best_elastic_net_bayes # Or grid_search_en.best_estimator_ or random_search_en.best_estimator_

# print("\nEntrenando el modelo ElasticNet final seleccionado en el conjunto de entrenamiento completo...")
# final_best_elastic_net_model.fit(X_train, y_train)
# print("Evaluando en el conjunto de prueba (X_test) con el modelo ElasticNet final...")
# y_pred_test_final_en = final_best_elastic_net_model.predict(X_test)

# # Calculate metrics on the test set with the final ElasticNet model
# test_mae_final_en = mean_absolute_error(y_test, y_pred_test_final_en)
# test_mse_final_en = mean_squared_error(y_test, y_pred_test_final_en)
# test_r2_final_en = r2_score(y_test, y_pred_test_final_en)
# test_mape_final_en = mean_absolute_percentage_error(y_test, y_pred_test_final_en)

# print("\nMétricas en el conjunto de prueba con el modelo ElasticNet final seleccionado:")
# print(f"  MAE:  {test_mae_final_en:.4f}")
# print(f"  MSE:  {test_mse_final_en:.4f}")
# print(f"  R2:   {test_r2_final_en:.4f}")
# print(f"  MAPE: {test_mape_final_en:.4f}%")

# KernelRidge

In [ ]:
# prompt: Implementa KernelRidge en Python con sklearn, optimiza alpha y gamma para kernel RBF usando GridSearchCV, RandomizedSearchCV y BayesSearchCV de skopt, y reporta MAE, MSE, R2 y MAPE con validación cruzada de 5 folds."
# Rango: alpha = [0.001, 0.01, 0.1], gamma = [0.001, 0.01] (Grid); alpha loguniform(0.001, 1), gamma loguniform(0.001, 1) (Random/Bayesian).
# Justificación: Alpha controla regularización, gamma ajusta escala del kernel RBF; rangos logarítmicos capturan no linealidades.

!pip install scikit-optimize

# Ensure scorers and kf are defined (as per the original code context)
# (assuming the previous cells defining these have been run)

from sklearn.kernel_ridge import KernelRidge


# Definir el modelo KernelRidge con kernel RBF
# Aumentar max_iter si es necesario para la convergencia (aunque KRR no tiene un solver iterativo como las lineales)
# kernel='rbf' especifica el tipo de kernel
kernel_ridge = KernelRidge(kernel='rbf')


# --- Optimization with GridSearchCV ---
print("\nIniciando optimización con GridSearchCV para KernelRidge (kernel RBF)...")

# Define parameter grid for GridSearchCV
# alpha = [0.001, 0.01, 0.1], gamma = [0.001, 0.01]
param_grid_krr = {
    'alpha': [0.001, 0.01, 0.1],
    'gamma': [0.001, 0.01]
}

grid_search_krr = GridSearchCV(estimator=kernel_ridge, param_grid=param_grid_krr,
                               scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                               cv=kf, verbose=1, n_jobs=-1)

start_time_grid_krr = time.time()
grid_search_krr.fit(X_train, y_train)
end_time_grid_krr = time.time()

print("\nResultados de GridSearchCV para KernelRidge:")
print(f"Mejores hiperparámetros encontrados: {grid_search_krr.best_params_}")
print(f"Mejor MAE promedio en validación: {-grid_search_krr.best_score_:.4f}")
print(f"Tiempo de ejecución de GridSearchCV: {end_time_grid_krr - start_time_grid_krr:.2f} segundos")

best_kernel_ridge_grid = grid_search_krr.best_estimator_

print("\nEvaluando el mejor modelo de GridSearchCV para KernelRidge con validación cruzada completa:")
cv_results_grid_best_krr = cross_validate(best_kernel_ridge_grid, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_grid_krr = -cv_results_grid_best_krr['test_mae']
cv_mse_grid_krr = -cv_results_grid_best_krr['test_mse']
cv_r2_grid_krr = cv_results_grid_best_krr['test_r2']
cv_mape_grid_krr = -cv_results_grid_best_krr['test_mape']

print(f"  MAE:  {cv_mae_grid_krr.mean():.4f} +/- {cv_mae_grid_krr.std():.4f}")
print(f"  MSE:  {cv_mse_grid_krr.mean():.4f} +/- {cv_mse_grid_krr.std():.4f}")
print(f"  R2:   {cv_r2_grid_krr.mean():.4f} +/- {cv_r2_grid_krr.std():.4f}")
print(f"  MAPE: {cv_mape_grid_krr.mean():.4f}% +/- {cv_mape_grid_krr.std():.4f}%")


# --- Optimization with RandomizedSearchCV ---
print("\nIniciando optimización con RandomizedSearchCV para KernelRidge (kernel RBF)...")

# Define parameter distribution for RandomizedSearchCV
# alpha loguniform(0.001, 1), gamma loguniform(0.001, 1)
param_dist_krr = {
    'alpha': loguniform(0.001, 1),
    'gamma': loguniform(0.001, 1)
}

# Number of iterations (adjust as needed)
n_iter_rand_krr = 50 # Example: try 50 random combinations

random_search_krr = RandomizedSearchCV(estimator=kernel_ridge, param_distributions=param_dist_krr,
                                       n_iter=n_iter_rand_krr,
                                       scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                                       cv=kf, verbose=1, random_state=42, n_jobs=-1) # random_state para reproducibilidad

start_time_rand_krr = time.time()
random_search_krr.fit(X_train, y_train)
end_time_rand_krr = time.time()

print("\nResultados de RandomizedSearchCV para KernelRidge:")
print(f"Mejores hiperparámetros encontrados: {random_search_krr.best_params_}")
print(f"Mejor MAE promedio en validación: {-random_search_krr.best_score_:.4f}")
print(f"Tiempo de ejecución de RandomizedSearchCV: {end_time_rand_krr - start_time_rand_krr:.2f} segundos")

best_kernel_ridge_rand = random_search_krr.best_estimator_

print("\nEvaluando el mejor modelo de RandomizedSearchCV para KernelRidge con validación cruzada completa:")
cv_results_rand_best_krr = cross_validate(best_kernel_ridge_rand, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_rand_krr = -cv_results_rand_best_krr['test_mae']
cv_mse_rand_krr = -cv_results_rand_best_krr['test_mse']
cv_r2_rand_krr = cv_results_rand_best_krr['test_r2']
cv_mape_rand_krr = -cv_results_rand_best_krr['test_mape']

print(f"  MAE:  {cv_mae_rand_krr.mean():.4f} +/- {cv_mae_rand_krr.std():.4f}")
print(f"  MSE:  {cv_mse_rand_krr.mean():.4f} +/- {cv_mse_rand_krr.std():.4f}")
print(f"  R2:   {cv_r2_rand_krr.mean():.4f} +/- {cv_r2_rand_krr.std():.4f}")
print(f"  MAPE: {cv_mape_rand_krr.mean():.4f}% +/- {cv_mape_rand_krr.std():.4f}%")


# --- Optimization with BayesSearchCV ---
print("\nIniciando optimización con BayesSearchCV para KernelRidge (kernel RBF)...")

# Define search spaces for BayesSearchCV
# alpha loguniform(0.001, 1), gamma loguniform(0.001, 1)
search_spaces_krr = {
    'alpha': Real(0.001, 1, prior='log-uniform'),
    'gamma': Real(0.001, 1, prior='log-uniform')
}

# Number of iterations (adjust as needed)
n_iter_bayes_krr = 50 # Example: try 50 iterations

bayes_search_krr = BayesSearchCV(estimator=kernel_ridge, search_spaces=search_spaces_krr,
                                 n_iter=n_iter_bayes_krr,
                                 scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                                 cv=kf, verbose=1, random_state=42, n_jobs=-1)

start_time_bayes_krr = time.time()
bayes_search_krr.fit(X_train, y_train)
end_time_bayes_krr = time.time()

print("\nResultados de BayesSearchCV para KernelRidge:")
print(f"Mejores hiperparámetros encontrados: {bayes_search_krr.best_params_}")
print(f"Mejor MAE promedio en validación: {-bayes_search_krr.best_score_:.4f}")
print(f"Tiempo de ejecución de BayesSearchCV: {end_time_bayes_krr - start_time_bayes_krr:.2f} segundos")

best_kernel_ridge_bayes = bayes_search_krr.best_estimator_

print("\nEvaluando el mejor modelo de BayesSearchCV para KernelRidge con validación cruzada completa:")
cv_results_bayes_best_krr = cross_validate(best_kernel_ridge_bayes, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_bayes_krr = -cv_results_bayes_best_krr['test_mae']
cv_mse_bayes_krr = -cv_results_bayes_best_krr['test_mse']
cv_r2_bayes_krr = cv_results_bayes_best_krr['test_r2']
cv_mape_bayes_krr = -cv_results_bayes_best_krr['test_mape']

print(f"  MAE:  {cv_mae_bayes_krr.mean():.4f} +/- {cv_mae_bayes_krr.std():.4f}")
print(f"  MSE:  {cv_mse_bayes_krr.mean():.4f} +/- {cv_mse_bayes_krr.std():.4f}")
print(f"  R2:   {cv_r2_bayes_krr.mean():.4f} +/- {cv_r2_bayes_krr.std():.4f}")
print(f"  MAPE: {cv_mape_bayes_krr.mean():.4f}% +/- {cv_mape_bayes_krr.std():.4f}%")


# --- Comparación de resultados de KernelRidge ---
print("\n--- Resumen de Resultados de Optimización para KernelRidge (MAE promedio en validación) ---")
print(f"GridSearchCV:      {-grid_search_krr.best_score_:.4f}")
print(f"RandomizedSearchCV: {-random_search_krr.best_score_:.4f}")
print(f"BayesSearchCV:      {-bayes_search_krr.best_score_:.4f}")

print("\n--- Reporte Completo de Métricas para KernelRidge (Promedio de 5-fold CV) ---")

print("\nGridSearchCV Mejor Modelo (KernelRidge):")
print(f"  MAE:  {cv_mae_grid_krr.mean():.4f}")
print(f"  MSE:  {cv_mse_grid_krr.mean():.4f}")
print(f"  R2:   {cv_r2_grid_krr.mean():.4f}")
print(f"  MAPE: {cv_mape_grid_krr.mean():.4f}%")

print("\nRandomizedSearchCV Mejor Modelo (KernelRidge):")
print(f"  MAE:  {cv_mae_rand_krr.mean():.4f}")
print(f"  MSE:  {cv_mse_rand_krr.mean():.4f}")
print(f"  R2:   {cv_r2_rand_krr.mean():.4f}")
print(f"  MAPE: {cv_mape_rand_krr.mean():.4f}%")

print("\nBayesSearchCV Mejor Modelo (KernelRidge):")
print(f"  MAE:  {cv_mae_bayes_krr.mean():.4f}")
print(f"  MSE:  {cv_mse_bayes_krr.mean():.4f}")
print(f"  R2:   {cv_r2_bayes_krr.mean():.4f}")
print(f"  MAPE: {cv_mape_bayes_krr.mean():.4f}%")

# You can select the overall best KernelRidge model based on validation MAE
# For instance, if BayesSearchCV gave the best result:
# final_best_kernel_ridge_model = best_kernel_ridge_bayes
# print("\nEl mejor modelo KernelRidge general basado en MAE promedio de validación es el de BayesSearchCV.")

# Optional: Train the chosen best KernelRidge model on the entire X_train and evaluate on X_test
# final_best_kernel_ridge_model = best_kernel_ridge_bayes # Or grid_search_krr.best_estimator_ or random_search_krr.best_estimator_

# print("\nEntrenando el modelo KernelRidge final seleccionado en el conjunto de entrenamiento completo...")
# final_best_kernel_ridge_model.fit(X_train, y_train)
# print("Evaluando en el conjunto de prueba (X_test) con el modelo KernelRidge final...")
# y_pred_test_final_krr = final_best_kernel_ridge_model.predict(X_test)

# # Calculate metrics on the test set with the final KernelRidge model
# test_mae_final_krr = mean_absolute_error(y_test, y_pred_test_final_krr)
# test_mse_final_krr = mean_squared_error(y_test, y_pred_test_final_krr)
# test_r2_final_krr = r2_score(y_test, y_pred_test_final_krr)
# test_mape_final_krr = mean_absolute_percentage_error(y_test, y_pred_test_final_krr)

# print("\nMétricas en el conjunto de prueba con el modelo KernelRidge final seleccionado:")
# print(f"  MAE:  {test_mae_final_krr:.4f}")
# print(f"  MSE:  {test_mse_final_krr:.4f}")
# print(f"  R2:   {test_r2_final_krr:.4f}")
# print(f"  MAPE: {test_mape_final_krr:.4f}%")



# SGDRegressor

In [ ]:
# prompt: Implementa SGDRegressor en Python con sklearn, optimiza alpha con learning_rate='invscaling' usando GridSearchCV, RandomizedSearchCV y BayesSearchCV de skopt, y reporta MAE, MSE, R2 y MAPE con validación cruzada de 5 folds."
# Rango: alpha = [0.0001, 0.001] (Grid); alpha loguniform(0.0001, 0.01) (Random/Bayesian).
# Justificación: Alpha regula regularización, learning_rate fijo para estabilidad; rangos pequeños para evitar overfitting.

from sklearn.linear_model import SGDRegressor

# Ensure scorers and kf are defined (as per the original code context)
# (assuming the previous cells defining these have been run)

# Define the SGDRegressor model
# Use learning_rate='invscaling' and set early_stopping=True for better performance and stability
sgd_regressor = SGDRegressor(learning_rate='invscaling', early_stopping=True,
                             random_state=42, max_iter=10000) # Increased max_iter

# --- Optimization with GridSearchCV ---
print("\nIniciando optimización con GridSearchCV para SGDRegressor...")

# Define parameter grid for GridSearchCV
# Range: alpha = [0.0001, 0.001]
param_grid_sgd = {
    'alpha': [0.0001, 0.001]
}

grid_search_sgd = GridSearchCV(estimator=sgd_regressor, param_grid=param_grid_sgd,
                               scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                               cv=kf, verbose=1, n_jobs=-1)

start_time_grid_sgd = time.time()
grid_search_sgd.fit(X_train, y_train)
end_time_grid_sgd = time.time()

print("\nResultados de GridSearchCV para SGDRegressor:")
print(f"Mejores hiperparámetros encontrados: {grid_search_sgd.best_params_}")
print(f"Mejor MAE promedio en validación: {-grid_search_sgd.best_score_:.4f}")
print(f"Tiempo de ejecución de GridSearchCV: {end_time_grid_sgd - start_time_grid_sgd:.2f} segundos")

best_sgd_grid = grid_search_sgd.best_estimator_

print("\nEvaluando el mejor modelo de GridSearchCV para SGDRegressor con validación cruzada completa:")
cv_results_grid_best_sgd = cross_validate(best_sgd_grid, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_grid_sgd = -cv_results_grid_best_sgd['test_mae']
cv_mse_grid_sgd = -cv_results_grid_best_sgd['test_mse']
cv_r2_grid_sgd = cv_results_grid_best_sgd['test_r2']
cv_mape_grid_sgd = -cv_results_grid_best_sgd['test_mape']

print(f"  MAE:  {cv_mae_grid_sgd.mean():.4f} +/- {cv_mae_grid_sgd.std():.4f}")
print(f"  MSE:  {cv_mse_grid_sgd.mean():.4f} +/- {cv_mse_grid_sgd.std():.4f}")
print(f"  R2:   {cv_r2_grid_sgd.mean():.4f} +/- {cv_r2_grid_sgd.std():.4f}")
print(f"  MAPE: {cv_mape_grid_sgd.mean():.4f}% +/- {cv_mape_grid_sgd.std():.4f}%")


# --- Optimization with RandomizedSearchCV ---
print("\nIniciando optimización con RandomizedSearchCV para SGDRegressor...")

# Define parameter distribution for RandomizedSearchCV
# Range: alpha loguniform(0.0001, 0.01)
param_dist_sgd = {
    'alpha': loguniform(0.0001, 0.01)
}

# Number of iterations (adjust as needed)
n_iter_rand_sgd = 50 # Example: try 50 random combinations

random_search_sgd = RandomizedSearchCV(estimator=sgd_regressor, param_distributions=param_dist_sgd,
                                       n_iter=n_iter_rand_sgd,
                                       scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                                       cv=kf, verbose=1, random_state=42, n_jobs=-1) # random_state para reproducibilidad

start_time_rand_sgd = time.time()
random_search_sgd.fit(X_train, y_train)
end_time_rand_sgd = time.time()

print("\nResultados de RandomizedSearchCV para SGDRegressor:")
print(f"Mejores hiperparámetros encontrados: {random_search_sgd.best_params_}")
print(f"Mejor MAE promedio en validación: {-random_search_sgd.best_score_:.4f}")
print(f"Tiempo de ejecución de RandomizedSearchCV: {end_time_rand_sgd - start_time_rand_sgd:.2f} segundos")

best_sgd_rand = random_search_sgd.best_estimator_

print("\nEvaluando el mejor modelo de RandomizedSearchCV para SGDRegressor con validación cruzada completa:")
cv_results_rand_best_sgd = cross_validate(best_sgd_rand, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_rand_sgd = -cv_results_rand_best_sgd['test_mae']
cv_mse_rand_sgd = -cv_results_rand_best_sgd['test_mse']
cv_r2_rand_sgd = cv_results_rand_best_sgd['test_r2']
cv_mape_rand_sgd = -cv_results_rand_best_sgd['test_mape']

print(f"  MAE:  {cv_mae_rand_sgd.mean():.4f} +/- {cv_mae_rand_sgd.std():.4f}")
print(f"  MSE:  {cv_mse_rand_sgd.mean():.4f} +/- {cv_mse_rand_sgd.std():.4f}")
print(f"  R2:   {cv_r2_rand_sgd.mean():.4f} +/- {cv_r2_rand_sgd.std():.4f}")
print(f"  MAPE: {cv_mape_rand_sgd.mean():.4f}% +/- {cv_mape_rand_sgd.std():.4f}%")


# --- Optimization with BayesSearchCV ---
print("\nIniciando optimización con BayesSearchCV para SGDRegressor...")

# Define search spaces for BayesSearchCV
# Range: alpha loguniform(0.0001, 0.01)
search_spaces_sgd = {
    'alpha': Real(0.0001, 0.01, prior='log-uniform')
}

# Number of iterations (adjust as needed)
n_iter_bayes_sgd = 50 # Example: try 50 iterations

bayes_search_sgd = BayesSearchCV(estimator=sgd_regressor, search_spaces=search_spaces_sgd,
                                 n_iter=n_iter_bayes_sgd,
                                 scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                                 cv=kf, verbose=1, random_state=42, n_jobs=-1)

start_time_bayes_sgd = time.time()
bayes_search_sgd.fit(X_train, y_train)
end_time_bayes_sgd = time.time()

print("\nResultados de BayesSearchCV para SGDRegressor:")
print(f"Mejores hiperparámetros encontrados: {bayes_search_sgd.best_params_}")
print(f"Mejor MAE promedio en validación: {-bayes_search_sgd.best_score_:.4f}")
print(f"Tiempo de ejecución de BayesSearchCV: {end_time_bayes_sgd - start_time_bayes_sgd:.2f} segundos")

best_sgd_bayes = bayes_search_sgd.best_estimator_

print("\nEvaluando el mejor modelo de BayesSearchCV para SGDRegressor con validación cruzada completa:")
cv_results_bayes_best_sgd = cross_validate(best_sgd_bayes, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_bayes_sgd = -cv_results_bayes_best_sgd['test_mae']
cv_mse_bayes_sgd = -cv_results_bayes_best_sgd['test_mse']
cv_r2_bayes_sgd = cv_results_bayes_best_sgd['test_r2']
cv_mape_bayes_sgd = -cv_results_bayes_best_sgd['test_mape']

print(f"  MAE:  {cv_mae_bayes_sgd.mean():.4f} +/- {cv_mae_bayes_sgd.std():.4f}")
print(f"  MSE:  {cv_mse_bayes_sgd.mean():.4f} +/- {cv_mse_bayes_sgd.std():.4f}")
print(f"  R2:   {cv_r2_bayes_sgd.mean():.4f} +/- {cv_r2_bayes_sgd.std():.4f}")
print(f"  MAPE: {cv_mape_bayes_sgd.mean():.4f}% +/- {cv_mape_bayes_sgd.std():.4f}%")


# --- Comparación de resultados de SGDRegressor ---
print("\n--- Resumen de Resultados de Optimización para SGDRegressor (MAE promedio en validación) ---")
print(f"GridSearchCV:      {-grid_search_sgd.best_score_:.4f}")
print(f"RandomizedSearchCV: {-random_search_sgd.best_score_:.4f}")
print(f"BayesSearchCV:      {-bayes_search_sgd.best_score_:.4f}")

print("\n--- Reporte Completo de Métricas para SGDRegressor (Promedio de 5-fold CV) ---")

print("\nGridSearchCV Mejor Modelo (SGDRegressor):")
print(f"  MAE:  {cv_mae_grid_sgd.mean():.4f}")
print(f"  MSE:  {cv_mse_grid_sgd.mean():.4f}")
print(f"  R2:   {cv_r2_grid_sgd.mean():.4f}")
print(f"  MAPE: {cv_mape_grid_sgd.mean():.4f}%")

print("\nRandomizedSearchCV Mejor Modelo (SGDRegressor):")
print(f"  MAE:  {cv_mae_rand_sgd.mean():.4f}")
print(f"  MSE:  {cv_mse_rand_sgd.mean():.4f}")
print(f"  R2:   {cv_r2_rand_sgd.mean():.4f}")
print(f"  MAPE: {cv_mape_rand_sgd.mean():.4f}%")

print("\nBayesSearchCV Mejor Modelo (SGDRegressor):")
print(f"  MAE:  {cv_mae_bayes_sgd.mean():.4f}")
print(f"  MSE:  {cv_mse_bayes_sgd.mean():.4f}")
print(f"  R2:   {cv_r2_bayes_sgd.mean():.4f}")
print(f"  MAPE: {cv_mape_bayes_sgd.mean():.4f}%")

# You can select the overall best SGDRegressor model based on validation MAE
# For instance, if BayesSearchCV gave the best result:
# final_best_sgd_model = best_sgd_bayes
# print("\nEl mejor modelo SGDRegressor general basado en MAE promedio de validación es el de BayesSearchCV.")

# Optional: Train the chosen best SGDRegressor model on the entire X_train and evaluate on X_test
# final_best_sgd_model = best_sgd_bayes # Or grid_search_sgd.best_estimator_ or random_search_sgd.best_estimator_

# print("\nEntrenando el modelo SGDRegressor final seleccionado en el conjunto de entrenamiento completo...")
# final_best_sgd_model.fit(X_train, y_train)
# print("Evaluando en el conjunto de prueba (X_test) con el modelo SGDRegressor final...")
# y_pred_test_final_sgd = final_best_sgd_model.predict(X_test)

# # Calculate metrics on the test set with the final SGDRegressor model
# test_mae_final_sgd = mean_absolute_error(y_test, y_pred_test_final_sgd)
# test_mse_final_sgd = mean_squared_error(y_test, y_pred_test_final_sgd)
# test_r2_final_sgd = r2_score(y_test, y_pred_test_final_sgd)
# test_mape_final_sgd = mean_absolute_percentage_error(y_test, y_pred_test_final_sgd)

# print("\nMétricas en el conjunto de prueba con el modelo SGDRegressor final seleccionado:")
# print(f"  MAE:  {test_mae_final_sgd:.4f}")
# print(f"  MSE:  {test_mse_final_sgd:.4f}")
# print(f"  R2:   {test_r2_final_sgd:.4f}")
# print(f"  MAPE: {test_mape_final_sgd:.4f}%")


# BayesianRidge

In [ ]:
# prompt: Implementa BayesianRidge en Python con sklearn, usa validación cruzada de 5 folds, y reporta MAE, MSE, R2 y MAPE, sin optimización de hiperparámetros."
# Rango: n_iter = [300] (Grid); no aplica para Random/Bayesian.
# Justificación: Robustez a multicolinealidad, parámetros predeterminados son suficientes.

import numpy as np
from sklearn.linear_model import BayesianRidge

# Ensure scorers and kf are defined (as per the original code context)
# (assuming the previous cells defining these have been run)

# Redefine the custom MAPE scorer if it wasn't in the preceding code or is needed again
def mean_absolute_percentage_error(y_true, y_pred):
    """Calcula el Error Porcentual Absoluto Medio (MAPE)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    epsilon = 1e-8
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# Create scorer objects for the metrics
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score, greater_is_better=True)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# Define the scoring dictionary for cross_validate
scoring = {
    'mae': mae_scorer,
    'mse': mse_scorer,
    'r2': r2_scorer,
    'mape': mape_scorer
}

# Define the BayesianRidge model with default parameters
# n_iter=300 as specified, but the prompt says "sin optimización de hiperparámetros",
# and n_iter doesn't apply to default sklearn BayesianRidge without tuning.
# Let's use the default parameters as requested by "sin optimización de hiperparámetros".
# If n_iter was a requirement, we'd clarify or set it, but the prompt suggests default.
# The default max_iter for BayesianRidge is 300, so it aligns if we were to set it,
# but sticking to "sin optimización" implies using the model as is.
model = BayesianRidge()

# Perform cross-validation with 5 folds (kf)
print("\nRealizando validación cruzada con BayesianRidge (parámetros predeterminados)...")

# Use cross_validate to get multiple metrics simultaneously
cv_results = cross_validate(model, X_train, y_train, cv=kf, scoring=scoring)

# Access the results arrays
# Error metrics (MAE, MSE, MAPE) are negative because make_scorer optimizes by maximizing.
# Take the absolute value for reporting.
cv_mae = -cv_results['test_mae']
cv_mse = -cv_results['test_mse']
cv_r2 = cv_results['test_r2']
cv_mape = -cv_results['test_mape']

# Report the average and standard deviation of each metric
print(f"\nResultados de Validación Cruzada para BayesianRidge (promedio +/- std dev):")
print(f"  MAE:  {cv_mae.mean():.4f} +/- {cv_mae.std():.4f}")
print(f"  MSE:  {cv_mse.mean():.4f} +/- {cv_mse.std():.4f}")
print(f"  R2:   {cv_r2.mean():.4f} +/- {cv_r2.std():.4f}")
print(f"  MAPE: {cv_mape.mean():.4f}% +/- {cv_mape.std():.4f}%")

# Optional: Train the model on the entire training set and evaluate on the test set
# This step is typically performed after model selection and hyperparameter tuning,
# but can be done here to show performance on unseen data with default parameters.
# print("\nEntrenando modelo final BayesianRidge en el conjunto de entrenamiento completo...")
# model.fit(X_train, y_train)
# print("Evaluando en el conjunto de prueba...")
# y_pred_test = model.predict(X_test)

# # Calculate metrics on the test set
# test_mae = mean_absolute_error(y_test, y_pred_test)
# test_mse = mean_squared_error(y_test, y_pred_test)
# test_r2 = r2_score(y_test, y_pred_test)
# test_mape = mean_absolute_percentage_error(y_test, y_pred_test)

# print("\nMétricas en el conjunto de prueba (BayesianRidge con parámetros predeterminados):")
# print(f"  MAE:  {test_mae:.4f}")
# print(f"  MSE:  {test_mse:.4f}")
# print(f"  R2:   {test_r2:.4f}")
# print(f"  MAPE: {test_mape:.4f}%")

# GaussianProcessRegressor

In [ ]:
# prompt: Implementa GaussianProcessRegressor en Python con sklearn, optimiza length_scale y alpha para kernel RBF usando GridSearchCV, RandomizedSearchCV y BayesSearchCV de skopt, y reporta MAE, MSE, R2 y MAPE con validación cruzada de 5 folds."
# # # Rango: length_scale = [0.1, 1], alpha = [0.001] (Grid); length_scale loguniform(0.1, 2), alpha loguniform(0.001, 0.1) (Random/Bayesian).
# # # Justificación: Length_scale ajusta correlación, alpha controla ruido; rangos logarítmicos para capturar variabilidad, alpha fijo pequeño para estabilidad.
# # Organízame el código para que al momento realizar la optimización de los hiperparámetros, no use tantos fits y no se demore

# Instalar scikit-optimize si no está instalado
!pip install scikit-optimize

import time
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real
from scipy.stats import loguniform

# Suponiendo que X_train, y_train, X_test, y_test están definidos
# Definir KFold para validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Definir métricas para evaluación
scoring_optimizer = {'mae': make_scorer(mean_absolute_error, greater_is_better=False)}
scoring = {
    'mae': make_scorer(mean_absolute_error, greater_is_better=False),
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'r2': make_scorer(r2_score, greater_is_better=True),
    'mape': make_scorer(mean_absolute_percentage_error, greater_is_better=False)
}

# Definir el modelo con menos reinicios
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (0.1, 2.0))
gpr = GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=5)

# --- Optimización con GridSearchCV ---
print("\nIniciando optimización con GridSearchCV...")
param_grid_gpr = {
    'kernel__k2__length_scale': [0.1, 1.0],
    'alpha': [0.001]
}
grid_search_gpr = GridSearchCV(estimator=gpr, param_grid=param_grid_gpr,
                               scoring=scoring_optimizer, refit='mae',
                               cv=kf, verbose=1, n_jobs=-1)
start_time_grid_gpr = time.time()
grid_search_gpr.fit(X_train, y_train)
end_time_grid_gpr = time.time()

print("\nResultados de GridSearchCV:")
print(f"Mejores hiperparámetros: {grid_search_gpr.best_params_}")
print(f"Mejor MAE promedio: {-grid_search_gpr.best_score_:.4f}")
print(f"Tiempo: {end_time_grid_gpr - start_time_grid_gpr:.2f} segundos")

# --- Optimización con RandomizedSearchCV ---
print("\nIniciando optimización con RandomizedSearchCV...")
param_dist_gpr = {
    'kernel__k2__length_scale': loguniform(0.1, 2.0),
    'alpha': loguniform(0.001, 0.1)
}
random_search_gpr = RandomizedSearchCV(estimator=gpr, param_distributions=param_dist_gpr,
                                      n_iter=10, scoring=scoring_optimizer, refit='mae',
                                      cv=kf, verbose=1, random_state=42, n_jobs=-1)
start_time_rand_gpr = time.time()
random_search_gpr.fit(X_train, y_train)
end_time_rand_gpr = time.time()

print("\nResultados de RandomizedSearchCV:")
print(f"Mejores hiperparámetros: {random_search_gpr.best_params_}")
print(f"Mejor MAE promedio: {-random_search_gpr.best_score_:.4f}")
print(f"Tiempo: {end_time_rand_gpr - start_time_rand_gpr:.2f} segundos")

# --- Optimización con BayesSearchCV ---
print("\nIniciando optimización con BayesSearchCV...")
search_spaces_gpr = {
    'kernel__k2__length_scale': Real(0.1, 2.0, prior='log-uniform'),
    'alpha': Real(0.001, 0.1, prior='log-uniform')
}
bayes_search_gpr = BayesSearchCV(estimator=gpr, search_spaces=search_spaces_gpr,
                                 n_iter=10, scoring=scoring_optimizer, refit='mae',
                                 cv=kf, verbose=1, random_state=42, n_jobs=-1)
start_time_bayes_gpr = time.time()
bayes_search_gpr.fit(X_train, y_train)
end_time_bayes_gpr = time.time()

print("\nResultados de BayesSearchCV:")
print(f"Mejores hiperparámetros: {bayes_search_gpr.best_params_}")
print(f"Mejor MAE promedio: {-bayes_search_gpr.best_score_:.4f}")
print(f"Tiempo: {end_time_bayes_gpr - start_time_bayes_gpr:.2f} segundos")

# Evaluación final del mejor modelo (BayesSearchCV) en el conjunto completo
best_gpr = bayes_search_gpr.best_estimator_
cv_results_best_gpr = cross_validate(best_gpr, X_train, y_train, cv=kf, scoring=scoring)

print("\nMétricas del mejor modelo (BayesSearchCV) con validación cruzada:")
print(f"  MAE:  {-cv_results_best_gpr['test_mae'].mean():.4f} +/- {cv_results_best_gpr['test_mae'].std():.4f}")
print(f"  MSE:  {-cv_results_best_gpr['test_mse'].mean():.4f} +/- {cv_results_best_gpr['test_mse'].std():.4f}")
print(f"  R2:   {cv_results_best_gpr['test_r2'].mean():.4f} +/- {cv_results_best_gpr['test_r2'].std():.4f}")
print(f"  MAPE: {-cv_results_best_gpr['test_mape'].mean():.4f}% +/- {cv_results_best_gpr['test_mape'].std():.4f}%")



# RandomForestRegressor

In [ ]:
# prompt: Implementa RandomForestRegressor en Python con sklearn, optimiza n_estimators, max_depth y min_samples_split usando GridSearchCV, RandomizedSearchCV y BayesSearchCV de skopt, y reporta MAE, MSE, R2 y MAPE con validación cruzada de 5 folds."
# Rango: n_estimators = [50, 256], max_depth = [None, 16], min_samples_split = [2, 10] (Grid); n_estimators randint(56, 256), max_depth [None] + list(range(5, 30)), min_samples_split randint(2, 25) (Random); n_estimators Integer(50, 256), max_depth Integer(5, 35), min_samples_split Integer(2, 25) (Bayesian).
# Justificación: n_estimators controla ensamblado, max_depth y min_samples_split evitan overfitting; rangos amplios para explorar complejidad.

# Importaciones necesarias
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from skopt import BayesSearchCV
from skopt.space import Integer
from scipy.stats import randint
import time
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

# Suponiendo que X_train, y_train están definidos
# Definir KFold para validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Definir métricas para evaluación
scoring_optimizer = {'mae': make_scorer(mean_absolute_error, greater_is_better=False)}
scoring = {
    'mae': make_scorer(mean_absolute_error, greater_is_better=False),
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'r2': make_scorer(r2_score, greater_is_better=True),
    'mape': make_scorer(mean_absolute_percentage_error, greater_is_better=False)
}

# Definir el modelo RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=42, n_jobs=-1)

# --- Optimización con GridSearchCV ---
print("\nIniciando optimización con GridSearchCV para RandomForestRegressor...")
param_grid_rf = {
    'n_estimators': [50, 100],  # Reducido para acelerar
    'max_depth': [None, 10],    # Reducido para acelerar
    'min_samples_split': [2, 5] # Reducido para acelerar
}
grid_search_rf = GridSearchCV(estimator=rf_regressor, param_grid=param_grid_rf,
                              scoring=scoring_optimizer, refit='mae',
                              cv=kf, verbose=1, n_jobs=-1)
start_time_grid_rf = time.time()
grid_search_rf.fit(X_train, y_train)
end_time_grid_rf = time.time()

print("\nResultados de GridSearchCV para RandomForestRegressor:")
print(f"Mejores hiperparámetros encontrados: {grid_search_rf.best_params_}")
print(f"Mejor MAE promedio en validación: {-grid_search_rf.best_score_:.4f}")
print(f"Tiempo de ejecución de GridSearchCV: {end_time_grid_rf - start_time_grid_rf:.2f} segundos")

# --- Optimización con RandomizedSearchCV ---
print("\nIniciando optimización con RandomizedSearchCV para RandomForestRegressor...")
param_dist_rf = {
    'n_estimators': randint(50, 150),  # Rango reducido
    'max_depth': [None] + list(range(5, 15)),  # Rango reducido
    'min_samples_split': randint(2, 10)  # Rango reducido
}
random_search_rf = RandomizedSearchCV(estimator=rf_regressor, param_distributions=param_dist_rf,
                                      n_iter=10,  # Reducido de 50 a 10
                                      scoring=scoring_optimizer, refit='mae',
                                      cv=kf, verbose=1, random_state=42, n_jobs=-1)
start_time_rand_rf = time.time()
random_search_rf.fit(X_train, y_train)
end_time_rand_rf = time.time()

print("\nResultados de RandomizedSearchCV para RandomForestRegressor:")
print(f"Mejores hiperparámetros encontrados: {random_search_rf.best_params_}")
print(f"Mejor MAE promedio en validación: {-random_search_rf.best_score_:.4f}")
print(f"Tiempo de ejecución de RandomizedSearchCV: {end_time_rand_rf - start_time_rand_rf:.2f} segundos")

# --- Optimización con BayesSearchCV ---
print("\nIniciando optimización con BayesSearchCV para RandomForestRegressor...")
search_spaces_rf = {
    'n_estimators': Integer(50, 150),  # Rango reducido
    'max_depth': Integer(5, 15),       # Rango reducido
    'min_samples_split': Integer(2, 10)  # Rango reducido
}
bayes_search_rf = BayesSearchCV(estimator=rf_regressor, search_spaces=search_spaces_rf,
                                n_iter=10,  # Reducido de 50 a 10
                                scoring=scoring_optimizer, refit='mae',
                                cv=kf, verbose=1, random_state=42, n_jobs=-1)
start_time_bayes_rf = time.time()
bayes_search_rf.fit(X_train, y_train)
end_time_bayes_rf = time.time()

print("\nResultados de BayesSearchCV para RandomForestRegressor:")
print(f"Mejores hiperparámetros encontrados: {bayes_search_rf.best_params_}")
print(f"Mejor MAE promedio en validación: {-bayes_search_rf.best_score_:.4f}")
print(f"Tiempo de ejecución de BayesSearchCV: {end_time_bayes_rf - start_time_bayes_rf:.2f} segundos")

# Evaluación final del mejor modelo (BayesSearchCV) en el conjunto completo
best_rf = bayes_search_rf.best_estimator_
cv_results_best_rf = cross_validate(best_rf, X_train, y_train, cv=kf, scoring=scoring)

print("\nMétricas del mejor modelo (BayesSearchCV) con validación cruzada:")
print(f"  MAE:  {-cv_results_best_rf['test_mae'].mean():.4f} +/- {cv_results_best_rf['test_mae'].std():.4f}")
print(f"  MSE:  {-cv_results_best_rf['test_mse'].mean():.4f} +/- {cv_results_best_rf['test_mse'].std():.4f}")
print(f"  R2:   {cv_results_best_rf['test_r2'].mean():.4f} +/- {cv_results_best_rf['test_r2'].std():.4f}")
print(f"  MAPE: {-cv_results_best_rf['test_mape'].mean():.4f}% +/- {cv_results_best_rf['test_mape'].std():.4f}%")


# SVR

In [ ]:
# prompt: "Implementa SVR en Python con sklearn, optimiza C, epsilon y gamma para kernel RBF usando GridSearchCV, RandomizedSearchCV y BayesSearchCV de skopt, y reporta MAE, MSE, R2 y MAPE con validación cruzada de 5 folds."

import numpy as np
from sklearn.svm import SVR
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import uniform, randint # Ensure randint is imported for Random Forest later if needed
from skopt.space import Real, Integer, Categorical # Ensure Integer and Categorical are imported if used later
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

# Ensure scorers and kf are defined (as per the original code context)
# (assuming the previous cells defining these have been run)

# Redefine the custom MAPE scorer if it wasn't in the preceding code or is needed again
def mean_absolute_percentage_error(y_true, y_pred):
    """Calcula el Error Porcentual Absoluto Medio (MAPE)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    epsilon = 1e-8
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# Create scorer objects for the metrics
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score, greater_is_better=True)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# Define the scoring dictionary for cross_validate and optimizers
# We will optimize based on MAE, so define a single scorer for the optimizers
scoring_optimizer = {'mae': mae_scorer}

# Define the full scoring dictionary for reporting results
scoring = {
    'mae': mae_scorer,
    'mse': mse_scorer,
    'r2': r2_scorer,
    'mape': mape_scorer
}


# Definir el modelo SVR con kernel RBF
svr = SVR(kernel='rbf')

# Assuming X_train, y_train, and kf are defined in previous cells

# --- Optimization with GridSearchCV ---
print("\nIniciando optimización con GridSearchCV para SVR (kernel RBF)...")

# Define parameter grid for GridSearchCV
# C = [1, 10, 100], epsilon = [0.01, 0.1, 0.5], gamma = ['scale', 'auto', 0.001]
param_grid_svr = {
    'C': [1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5],
    'gamma': ['scale', 'auto', 0.001] # 'scale' and 'auto' are also valid values
}

grid_search_svr = GridSearchCV(estimator=svr, param_grid=param_grid_svr,
                               scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                               cv=kf, verbose=1, n_jobs=-1)

start_time_grid_svr = time.time()
grid_search_svr.fit(X_train, y_train)
end_time_grid_svr = time.time()

print("\nResultados de GridSearchCV para SVR:")
print(f"Mejores hiperparámetros encontrados: {grid_search_svr.best_params_}")
print(f"Mejor MAE promedio en validación: {-grid_search_svr.best_score_:.4f}")
print(f"Tiempo de ejecución de GridSearchCV: {end_time_grid_svr - start_time_grid_svr:.2f} segundos")

best_svr_grid = grid_search_svr.best_estimator_

print("\nEvaluando el mejor modelo de GridSearchCV para SVR con validación cruzada completa:")
cv_results_grid_best_svr = cross_validate(best_svr_grid, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_grid_svr = -cv_results_grid_best_svr['test_mae']
cv_mse_grid_svr = -cv_results_grid_best_svr['test_mse']
cv_r2_grid_svr = cv_results_grid_best_svr['test_r2']
cv_mape_grid_svr = -cv_results_grid_best_svr['test_mape']

print(f"  MAE:  {cv_mae_grid_svr.mean():.4f} +/- {cv_mae_grid_svr.std():.4f}")
print(f"  MSE:  {cv_mse_grid_svr.mean():.4f} +/- {cv_mse_grid_svr.std():.4f}")
print(f"  R2:   {cv_r2_grid_svr.mean():.4f} +/- {cv_r2_grid_svr.std():.4f}")
print(f"  MAPE: {cv_mape_grid_svr.mean():.4f}% +/- {cv_mape_grid_svr.std():.4f}%")


# --- Optimization with RandomizedSearchCV ---
print("\nIniciando optimización con RandomizedSearchCV para SVR (kernel RBF)...")

# Define parameter distribution for RandomizedSearchCV
# C loguniform(1, 1000), epsilon uniform(0, 1), gamma loguniform(0.0001, 1)
param_dist_svr = {
    'C': loguniform(1, 1000),
    'epsilon': uniform(0, 1),
    'gamma': loguniform(0.0001, 1)
}

# Number of iterations (puntos a probar). Ajusta este valor según el tiempo disponible.
n_iter_rand_svr = 100 # Example: try 100 random combinations

random_search_svr = RandomizedSearchCV(estimator=svr, param_distributions=param_dist_svr,
                                       n_iter=n_iter_rand_svr,
                                       scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                                       cv=kf, verbose=1, random_state=42, n_jobs=-1) # random_state para reproducibilidad

start_time_rand_svr = time.time()
random_search_svr.fit(X_train, y_train)
end_time_rand_svr = time.time()

print("\nResultados de RandomizedSearchCV para SVR:")
print(f"Mejores hiperparámetros encontrados: {random_search_svr.best_params_}")
print(f"Mejor MAE promedio en validación: {-random_search_svr.best_score_:.4f}")
print(f"Tiempo de ejecución de RandomizedSearchCV: {end_time_rand_svr - start_time_rand_svr:.2f} segundos")

best_svr_rand = random_search_svr.best_estimator_

print("\nEvaluando el mejor modelo de RandomizedSearchCV para SVR con validación cruzada completa:")
cv_results_rand_best_svr = cross_validate(best_svr_rand, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_rand_svr = -cv_results_rand_best_svr['test_mae']
cv_mse_rand_svr = -cv_results_rand_best_svr['test_mse']
cv_r2_rand_svr = cv_results_rand_best_svr['test_r2']
cv_mape_rand_svr = -cv_results_rand_best_svr['test_mape']

print(f"  MAE:  {cv_mae_rand_svr.mean():.4f} +/- {cv_mae_rand_svr.std():.4f}")
print(f"  MSE:  {cv_mse_rand_svr.mean():.4f} +/- {cv_mse_rand_svr.std():.4f}")
print(f"  R2:   {cv_r2_rand_svr.mean():.4f} +/- {cv_r2_rand_svr.std():.4f}")
print(f"  MAPE: {cv_mape_rand_svr.mean():.4f}% +/- {cv_mape_rand_svr.std():.4f}%")


# --- Optimization with BayesSearchCV ---
print("\nIniciando optimización con BayesSearchCV para SVR (kernel RBF)...")

# Define search spaces for BayesSearchCV
# C Real(1e-1, 1e3, prior='log-uniform'), epsilon Real(1e-3, 1e0, prior='uniform'), gamma Real(1e-4, 1e0, prior='log-uniform')
search_spaces_svr = {
    'C': Real(1e-1, 1e3, prior='log-uniform'),
    'epsilon': Real(1e-3, 1e0, prior='uniform'),
    'gamma': Real(1e-4, 1e0, prior='log-uniform')
}

# Number of iterations (puntos a explorar). Generalmente requiere menos que RandomizedSearch.
n_iter_bayes_svr = 100 # Example: try 100 iterations. Adjust based on computational resources.

bayes_search_svr = BayesSearchCV(estimator=svr, search_spaces=search_spaces_svr,
                                 n_iter=n_iter_bayes_svr,
                                 scoring=scoring_optimizer, refit='mae', # Optimize using MAE
                                 cv=kf, verbose=1, random_state=42, n_jobs=-1)

start_time_bayes_svr = time.time()
bayes_search_svr.fit(X_train, y_train)
end_time_bayes_svr = time.time()

print("\nResultados de BayesSearchCV para SVR:")
print(f"Mejores hiperparámetros encontrados: {bayes_search_svr.best_params_}")
print(f"Mejor MAE promedio en validación: {-bayes_search_svr.best_score_:.4f}")
print(f"Tiempo de ejecución de BayesSearchCV: {end_time_bayes_svr - start_time_bayes_svr:.2f} segundos")

best_svr_bayes = bayes_search_svr.best_estimator_

print("\nEvaluando el mejor modelo de BayesSearchCV para SVR con validación cruzada completa:")
cv_results_bayes_best_svr = cross_validate(best_svr_bayes, X_train, y_train, cv=kf, scoring=scoring)

cv_mae_bayes_svr = -cv_results_bayes_best_svr['test_mae']
cv_mse_bayes_svr = -cv_results_bayes_best_svr['test_mse']
cv_r2_bayes_svr = cv_results_bayes_best_svr['test_r2']
cv_mape_bayes_svr = -cv_results_bayes_best_svr['test_mape']

print(f"  MAE:  {cv_mae_bayes_svr.mean():.4f} +/- {cv_mae_bayes_svr.std():.4f}")
print(f"  MSE:  {cv_mse_bayes_svr.mean():.4f} +/- {cv_mse_bayes_svr.std():.4f}")
print(f"  R2:   {cv_r2_bayes_svr.mean():.4f} +/- {cv_r2_bayes_svr.std():.4f}")
print(f"  MAPE: {cv_mape_bayes_svr.mean():.4f}% +/- {cv_mape_bayes_svr.std():.4f}%")


# --- Comparación de resultados de SVR ---
print("\n--- Resumen de Resultados de Optimización para SVR (MAE promedio en validación) ---")
print(f"GridSearchCV:      {-grid_search_svr.best_score_:.4f}")
print(f"RandomizedSearchCV: {-random_search_svr.best_score_:.4f}")
print(f"BayesSearchCV:      {-bayes_search_svr.best_score_:.4f}")

print("\n--- Reporte Completo de Métricas para SVR (Promedio de 5-fold CV) ---")

print("\nGridSearchCV Mejor Modelo (SVR):")
print(f"  MAE:  {cv_mae_grid_svr.mean():.4f}")
print(f"  MSE:  {cv_mse_grid_svr.mean():.4f}")
print(f"  R2:   {cv_r2_grid_svr.mean():.4f}")
print(f"  MAPE: {cv_mape_grid_svr.mean():.4f}%")

print("\nRandomizedSearchCV Mejor Modelo (SVR):")
print(f"  MAE:  {cv_mae_rand_svr.mean():.4f}")
print(f"  MSE:  {cv_mse_rand_svr.mean():.4f}")
print(f"  R2:   {cv_r2_rand_svr.mean():.4f}")
print(f"  MAPE: {cv_mape_rand_svr.mean():.4f}%")

print("\nBayesSearchCV Mejor Modelo (SVR):")
print(f"  MAE:  {cv_mae_bayes_svr.mean():.4f}")
print(f"  MSE:  {cv_mse_bayes_svr.mean():.4f}")
print(f"  R2:   {cv_r2_bayes_svr.mean():.4f}")
print(f"  MAPE: {cv_mape_bayes_svr.mean():.4f}%")

# You can select the overall best SVR model based on validation MAE
# For instance, if BayesSearchCV gave the best result:
# final_best_svr_model = best_svr_bayes
# print("\nEl mejor modelo SVR general basado en MAE promedio de validación es el de BayesSearchCV.")

# Optional: Train the chosen best SVR model on the entire X_train and evaluate on X_test
# final_best_svr_model = best_svr_bayes # Or grid_search_svr.best_estimator_ or random_search_svr.best_estimator_

# print("\nEntrenando el modelo SVR final seleccionado en el conjunto de entrenamiento completo...")
# final_best_svr_model.fit(X_train, y_train)
# print("Evaluando en el conjunto de prueba (X_test) con el modelo SVR final...")
# y_pred_test_final_svr = final_best_svr_model.predict(X_test)

# # Calculate metrics on the test set with the final SVR model
# test_mae_final_svr = mean_absolute_error(y_test, y_pred_test_final_svr)
# test_mse_final_svr = mean_squared_error(y_test, y_pred_test_final_svr)
# test_r2_final_svr = r2_score(y_test, y_pred_test_final_svr)
# test_mape_final_svr = mean_absolute_percentage_error(y_test, y_pred_test_final_svr)

# print("\nMétricas en el conjunto de prueba con el modelo SVR final seleccionado:")
# print(f"  MAE:  {test_mae_final_svr:.4f}")
# print(f"  MSE:  {test_mse_final_svr:.4f}")
# print(f"  R2:   {test_r2_final_svr:.4f}")
# print(f"  MAPE: {test_mape_final_svr:.4f}%")

# Resultados

In [ ]:
# prompt: Crea gráficas de barras que comparen las métricas MAE, MSE, R2 y MAPE de cada uno de los modelos.
# Indica los tres mejores modelos según los rendimientos de cada uno y para los tres mejores modelos, genera gráficas de importancia de características.
# Asegúrate de que todas las gráficas sean claras, bien etiquetadas y adecuadas para un dashboard de Streamlit.

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import BayesianRidge, SGDRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from scipy.stats import randint
from sklearn.linear_model import LinearRegression # Import LinearRegression for coefficient plotting

# Dictionary to store the best results for each model (optimized)
model_results = {
    'Linear Regression (Default)': {
        'MAE': cv_mae.mean(),
        'MSE': cv_mse.mean(),
        'R2': cv_r2.mean(),
        'MAPE': cv_mape.mean()
    },
    'Lasso (Bayes)': { # Using BayesSearchCV as it often finds better solutions
        'MAE': cv_mae_bayes.mean(),
        'MSE': cv_mse_bayes.mean(),
        'R2': cv_r2_bayes.mean(),
        'MAPE': cv_mape_bayes.mean()
    },
    'ElasticNet (Bayes)': { # Using BayesSearchCV
        'MAE': cv_mae_bayes_en.mean(),
        'MSE': cv_mse_bayes_en.mean(),
        'R2': cv_r2_bayes_en.mean(),
        'MAPE': cv_mape_bayes_en.mean()
    },
    'KernelRidge (Bayes)': { # Using BayesSearchCV
        'MAE': cv_mae_bayes_krr.mean(),
        'MSE': cv_mse_bayes_krr.mean(),
        'R2': cv_r2_bayes_krr.mean(),
        'MAPE': cv_mape_bayes_krr.mean()
    },
    'SGDRegressor (Bayes)': { # Using BayesSearchCV
        'MAE': cv_mae_bayes_sgd.mean(),
        'MSE': cv_mse_bayes_sgd.mean(),
        'R2': cv_r2_bayes_sgd.mean(),
        'MAPE': cv_mape_bayes_sgd.mean()
    },
     'BayesianRidge (Default)': { # As this model was run without optimization
        'MAE': cv_mae.mean(), # These variables should hold the results from the last BayesianRidge run
        'MSE': cv_mse.mean(),
        'R2': cv_r2.mean(),
        'MAPE': cv_mape.mean()
    },
    'GaussianProcessRegressor (Bayes)': { # Using BayesSearchCV
        'MAE': -cv_results_best_gpr['test_mae'].mean(),
        'MSE': -cv_results_best_gpr['test_mse'].mean(),
        'R2': cv_results_best_gpr['test_r2'].mean(),
        'MAPE': -cv_results_best_gpr['test_mape'].mean()
    },
    'RandomForestRegressor (Bayes)': { # Using BayesSearchCV
        'MAE': -cv_results_best_rf['test_mae'].mean(),
        'MSE': -cv_results_best_rf['test_mse'].mean(),
        'R2': cv_results_best_rf['test_r2'].mean(),
        'MAPE': -cv_results_best_rf['test_mape'].mean()
    },
    'SVR (Bayes)': { # Using BayesSearchCV
        'MAE': -cv_results_bayes_best_svr['test_mae'].mean(),
        'MSE': -cv_results_bayes_best_svr['test_mse'].mean(),
        # Corrected: Access 'test_r2' from the dictionary
        'R2': cv_results_bayes_best_svr['test_r2'].mean(),
        'MAPE': cv_mape_bayes_svr.mean() # The variable name was slightly different
    }
}

# Convert results to DataFrame for easy plotting
results_df = pd.DataFrame.from_dict(model_results, orient='index')

# Sort models by MAE (lower is better)
results_df_sorted_mae = results_df.sort_values(by='MAE')
print("\nRanking de Modelos por MAE:")
print(results_df_sorted_mae[['MAE']])

# Sort models by MSE (lower is better)
results_df_sorted_mse = results_df.sort_values(by='MSE')
print("\nRanking de Modelos por MSE:")
print(results_df_sorted_mse[['MSE']])

# Sort models by R2 (higher is better)
results_df_sorted_r2 = results_df.sort_values(by='R2', ascending=False)
print("\nRanking de Modelos por R2:")
print(results_df_sorted_r2[['R2']])

# Sort models by MAPE (lower is better)
results_df_sorted_mape = results_df.sort_values(by='MAPE')
print("\nRanking de Modelos por MAPE:")
print(results_df_sorted_mape[['MAPE']])


# Define the best models based on MAE (example, adjust if criteria changes)
best_mae_models = results_df_sorted_mae.head(3).index.tolist()
print(f"\nLos 3 mejores modelos según MAE son: {best_mae_models}")

# Define the best models based on MSE (example, adjust if criteria changes)
best_mse_models = results_df_sorted_mse.head(3).index.tolist()
print(f"\nLos 3 mejores modelos según MSE son: {best_mse_models}")

# Define the best models based on R2 (example, adjust if criteria changes)
best_r2_models = results_df_sorted_r2.head(3).index.tolist()
print(f"\nLos 3 mejores modelos según R2 son: {best_r2_models}")

# Define the best models based on MAPE (example, adjust if criteria changes)
best_mape_models = results_df_sorted_mape.head(3).index.tolist()
print(f"\nLos 3 mejores modelos según MAPE son: {best_mape_models}")


# Combine the top models from each metric (optional, to get a broader view)
all_top_models = list(set(best_mae_models + best_mse_models + best_r2_models + best_mape_models))
print(f"\nModelos que aparecieron en el top 3 de al menos una métrica: {all_top_models}")

# For the purpose of feature importance visualization, let's select the top 3 models based on MAE
top_3_models_for_importance = best_mae_models
print(f"\nSe generarán gráficas de importancia de características para los 3 mejores modelos según MAE: {top_3_models_for_importance}")


# Plotting function for metrics
def plot_metrics_bar(results_df, metric, title, ylabel, ascending=True):
    plt.figure(figsize=(12, 6))
    sns.barplot(x=results_df.index, y=metric, data=results_df.sort_values(by=metric, ascending=ascending))
    plt.title(title, fontsize=16)
    plt.xlabel("Modelo", fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Generate bar plots for each metric
plot_metrics_bar(results_df, 'MAE', 'Comparación de Modelos por MAE', 'MAE', ascending=True)
plot_metrics_bar(results_df, 'MSE', 'Comparación de Modelos por MSE', 'MSE', ascending=True)
plot_metrics_bar(results_df, 'R2', 'Comparación de Modelos por R2', 'R2 Score', ascending=False)
plot_metrics_bar(results_df, 'MAPE', 'Comparación de Modelos por MAPE', 'MAPE (%)', ascending=True)

# Function to get the best estimator for a given model name (optimized via BayesSearchCV if available)
def get_best_estimator(model_name):
    # Make sure to have the optimizer objects defined in the environment if needed
    # (e.g., bayes_search, bayes_search_en, etc.)
    if model_name == 'Linear Regression (Default)':
        return LinearRegression().fit(X_train, y_train) # Train default Linear Regression for coefficients
    elif model_name == 'Lasso (Bayes)':
        # Assuming bayes_search is the BayesSearchCV object for Lasso
        return bayes_search.best_estimator_
    elif model_name == 'ElasticNet (Bayes)':
         # Assuming bayes_search_en is the BayesSearchCV object for ElasticNet
        return bayes_search_en.best_estimator_
    elif model_name == 'KernelRidge (Bayes)':
         # KRR does not have feature coefficients like linear models or tree-based models
         print(f"Advertencia: {model_name} no proporciona importancia de características directa (coeficientes).")
         return None
    elif model_name == 'SGDRegressor (Bayes)':
         # Assuming bayes_search_sgd is the BayesSearchCV object for SGDRegressor
        return bayes_search_sgd.best_estimator_
    elif model_name == 'BayesianRidge (Default)':
        # Train a default BayesianRidge model for coefficients
        return BayesianRidge().fit(X_train, y_train)
    elif model_name == 'GaussianProcessRegressor (Bayes)':
        # GPR does not have feature importance based on coefficients or tree structure
        print(f"Advertencia: {model_name} no proporciona importancia de características directa.")
        return None
    elif model_name == 'RandomForestRegressor (Bayes)':
        # Assuming bayes_search_rf is the BayesSearchCV object for RandomForestRegressor
        return bayes_search_rf.best_estimator_
    elif model_name == 'SVR (Bayes)':
        # SVR with RBF kernel does not provide linear coefficients
        print(f"Advertencia: {model_name} no proporciona importancia de características directa.")
        return None
    else:
        return None


# Generate Feature Importance plots for the top 3 models (based on MAE)
print("\nGenerando gráficas de importancia de características para los 3 mejores modelos (según MAE):")

for model_name in top_3_models_for_importance:
    print(f"\nImportancia de Características para: {model_name}")
    best_model = get_best_estimator(model_name)

    if best_model:
        if hasattr(best_model, 'coef_'): # For linear models (LinearRegression, Lasso, ElasticNet, SGDRegressor, BayesianRidge)
            # Ensure X_train is a pandas DataFrame with named columns for plotting
            if isinstance(X_train, pd.DataFrame):
                coefficients = pd.Series(best_model.coef_, index=X_train.columns)
                # Sort coefficients by absolute value for importance
                sorted_coefficients = coefficients.abs().sort_values(ascending=False)

                plt.figure(figsize=(10, min(20, len(sorted_coefficients) * 0.3))) # Adjust figure size based on num features
                sns.barplot(x=sorted_coefficients.values, y=sorted_coefficients.index)
                plt.title(f'Importancia de Características (Coeficientes Absolutos) - {model_name}', fontsize=14)
                plt.xlabel('Magnitud del Coeficiente', fontsize=12)
                plt.ylabel('Característica', fontsize=12)
                plt.tight_layout()
                plt.show()
            else:
                 print(f"Advertencia: X_train no es un DataFrame. No se puede generar la gráfica de importancia de características para {model_name}.")


        elif hasattr(best_model, 'feature_importances_'): # For tree-based models (RandomForestRegressor)
             # Ensure X_train is a pandas DataFrame with named columns for plotting
            if isinstance(X_train, pd.DataFrame):
                importances = pd.Series(best_model.feature_importances_, index=X_train.columns)
                # Sort feature importances
                sorted_importances = importances.sort_values(ascending=False)

                plt.figure(figsize=(10, min(20, len(sorted_importances) * 0.3))) # Adjust figure size
                sns.barplot(x=sorted_importances.values, y=sorted_importances.index)
                plt.title(f'Importancia de Características - {model_name}', fontsize=14)
                plt.xlabel('Importancia', fontsize=12)
                plt.ylabel('Característica', fontsize=12)
                plt.tight_layout()
                plt.show()
            else:
                 print(f"Advertencia: X_train no es un DataFrame. No se puede generar la gráfica de importancia de características para {model_name}.")
        else:
             print(f"El modelo {model_name} no tiene atributos 'coef_' o 'feature_importances_'.")
    else:
        print(f"No se pudo obtener el mejor estimador para {model_name} o no aplica para importancia de características.")


# Print the top 3 models based on each metric explicitly again
print("\n--- Resumen de los 3 Mejores Modelos por Métrica ---")
print("Según MAE (menor es mejor):")
print(results_df_sorted_mae.head(3)[['MAE']])
print("\nSegún MSE (menor es mejor):")
print(results_df_sorted_mse.head(3)[['MSE']])
print("\nSegún R2 (mayor es mejor):")
print(results_df_sorted_r2.head(3)[['R2']])
print("\nSegún MAPE (menor es mejor):")
print(results_df_sorted_mape.head(3)[['MAPE']])

# DashBoard

In [ ]:
!pip install scikit-optimize
!pip install streamlit -q #instalación de librerías
!pip install pyngrok
!pip install optuna
!pip install streamlit pandas matplotlib seaborn scikit-learn pyngrok kagglehub
!pip install pyngrok streamlit --quiet

In [ ]:
!mkdir pages

In [ ]:
%%writefile TAM.py
import streamlit as st
from pyngrok import ngrok
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import shap
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, r2_score
import optuna
from optuna.samplers import TPESampler
from scipy.stats import loguniform, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
# prompt: Histograma de "SalePrice" con sns.histplot(df['SalePrice'], kde=True).
# Mapa de calor de correlación con sns.heatmap(df.corr(), annot=True, cmap='coolwarm').
# Gráficos de barras para variables categóricas, como sns.countplot(x='Neighborhood', data=df).
# Usa st.pyplot() para mostrar estas gráficas en el dashboard.

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from scipy.stats import loguniform, uniform, randint

# Configuración de la página de Streamlit
st.set_page_config(layout="wide", page_title="Análisis y Modelización de Precios de Viviendas")

st.title('Análisis y Modelización de Precios de Viviendas')

# Cargar los datos
@st.cache_data
def load_data():
    try:
        df = pd.read_csv('/content/drive/Shareddrives/UNAL_Colab/Teoría de Aprendizaje de Máquina/AmesHousing.csv')
        # Limpiar nombres de columnas (quitar espacios y caracteres especiales)
        df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=True)
        return df
    except FileNotFoundError:
        st.error("Error: Asegúrese de que el archivo 'AmesHousing.csv' esté en la ruta especificada.")
        return pd.DataFrame()

df = load_data()

if df.empty:
    st.stop()

st.subheader("Datos Originales (primeras 5 filas)")
st.dataframe(df.head())

# Procesamiento de Datos (Basado en tu código anterior)
@st.cache_data
def preprocess_data(df):
    # Copia del dataframe para evitar modificar el original en caché
    df_processed = df.copy()

    # Eliminar columnas con muchos valores nulos
    threshold = len(df_processed) * 0.5 # Eliminar columnas con más del 50% de nulos
    cols_to_drop_nulls = df_processed.columns[df_processed.isnull().sum() > threshold].tolist()
    df_processed = df_processed.drop(columns=cols_to_drop_nulls)
    st.sidebar.write(f"Columnas eliminadas por exceso de nulos (>50%): {cols_to_drop_nulls}")

    # Imputación de valores nulos
    # Identificar columnas por tipo de dato después de eliminar las columnas con muchos nulos
    numerical_cols = df_processed.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = df_processed.select_dtypes(include='object').columns.tolist()

    # Imputar numéricas con la mediana
    for col in numerical_cols:
        if df_processed[col].isnull().any():
            median_val = df_processed[col].median()
            df_processed[col].fillna(median_val, inplace=True)

    # Imputar categóricas con la moda o una categoría 'missing'
    for col in categorical_cols:
        if df_processed[col].isnull().any():
            # For simplicity, impute with a placeholder 'Missing' or the mode
            # Here using 'Missing' as it often preserves info about the missingness
            df_processed[col].fillna('Missing', inplace=True)

    # Crear características de edad (ejemplo: edad de la casa y edad de remodelación)
    if 'YearBuilt' in df_processed.columns:
         current_year = 2023 # O el año actual relevante
         df_processed['Age'] = current_year - df_processed['YearBuilt']
         numerical_cols.append('Age') # Añadir la nueva columna numérica
         # Eliminar YearBuilt para evitar colinealidad
         if 'YearBuilt' in df_processed.columns:
             df_processed = df_processed.drop(columns=['YearBuilt'])
             if 'YearBuilt' in numerical_cols:
                 numerical_cols.remove('YearBuilt')

    if 'YearRemodAdd' in df_processed.columns:
         df_processed['YearsSinceRemodel'] = current_year - df_processed['YearRemodAdd']
         numerical_cols.append('YearsSinceRemodel') # Añadir la nueva columna numérica
         # Eliminar YearRemodAdd para evitar colinealidad
         if 'YearRemodAdd' in df_processed.columns:
             df_processed = df_processed.drop(columns=['YearRemodAdd'])
             if 'YearRemodAdd' in numerical_cols:
                 numerical_cols.remove('YearRemodAdd')


    # Identificar variables categóricas (nominales y ordinales si aplica)
    # Aquí, para simplificar, trataremos todas las 'object' como nominales para One-Hot Encoding
    # Si tuvieras un diccionario de mapeo para ordinales, lo aplicarías aquí antes de One-Hot Encoding

    # Separar SalePrice antes de la codificación
    if 'SalePrice' in df_processed.columns:
        y = df_processed['SalePrice']
        X = df_processed.drop('SalePrice', axis=1)
    else:
        st.error("La columna 'SalePrice' no se encontró en el DataFrame procesado.")
        return None, None, None # Retorna None si SalePrice no está presente


    # Aplicar One-Hot Encoding a las columnas categóricas restantes en X
    X = pd.get_dummies(X, columns=categorical_cols, dummy_na=False) # dummy_na=False si ya imputaste nulos

    # Asegurarse de que X solo contenga columnas numéricas después del get_dummies
    X = X.select_dtypes(include=np.number)

    # Asegurarse de que y coincida con las filas de X después de cualquier posible eliminación de filas (si hubo dropna)
    # Aunque con la imputación no debería haber NaNs en X, es una buena práctica.
    # Si hubo alguna eliminación de filas por NaNs en X antes, necesitarías re-indexar y.
    # Dado que imputamos, no esperamos NaNs restantes, pero si los hubiera, se podría añadir X = X.dropna()

    # Retornar X y y
    return X, y, df_processed # También retornamos el df_processed por si se necesita para visualizaciones pre-split

X, y, df_processed = preprocess_data(df.copy()) # Usar una copia para el procesamiento

if X is None or y is None:
     st.stop()

st.sidebar.subheader("Opciones de Visualización")
viz_option = st.sidebar.selectbox("Selecciona una visualización:",
                                   ["Distribución de SalePrice",
                                    "Mapa de Calor de Correlación",
                                    "Relación con SalePrice (Scatter plots)",
                                    "Distribución de Variables Categóricas (Original)",
                                    "Distribución de Variables Numéricas"])

# Visualizaciones
if viz_option == "Distribución de SalePrice":
    st.subheader("Distribución de SalePrice")
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Histograma de SalePrice
    sns.histplot(df_processed['SalePrice'], kde=True, ax=axes[0])
    axes[0].set_title('Distribución de SalePrice')
    axes[0].set_xlabel('SalePrice')
    axes[0].set_ylabel('Frecuencia')

    # Boxplot de SalePrice
    sns.boxplot(y=df_processed['SalePrice'], ax=axes[1])
    axes[1].set_title('Boxplot de SalePrice')
    axes[1].set_ylabel('SalePrice')

    plt.tight_layout()
    st.pyplot(fig)

elif viz_option == "Mapa de Calor de Correlación":
    st.subheader("Mapa de Calor de Correlación")
    numeric_df_for_corr = df_processed.select_dtypes(include=np.number)

    if 'SalePrice' in numeric_df_for_corr.columns:
        st.write("Correlación de variables numéricas con SalePrice:")
        correlation_with_saleprice = numeric_df_for_corr.corr()['SalePrice'].sort_values(ascending=False)
        st.dataframe(correlation_with_saleprice) # Mostrar la tabla de correlación

        # Visualizar el heatmap completo (puede ser muy denso, opcional)
        # st.write("Heatmap de Matriz de Correlación Completa (Variables Numéricas):")
        # fig_full_corr, ax_full_corr = plt.subplots(figsize=(15, 12))
        # sns.heatmap(numeric_df_for_corr.corr(), cmap='coolwarm', annot=False, ax=ax_full_corr)
        # ax_full_corr.set_title('Matriz de Correlación de Variables Numéricas')
        # st.pyplot(fig_full_corr)

        # Visualizar solo las correlaciones con SalePrice (más legible)
        st.write("Heatmap de Correlación con SalePrice:")
        fig_saleprice_corr, ax_saleprice_corr = plt.subplots(figsize=(8, 10))
        sns.heatmap(correlation_with_saleprice.to_frame(), cmap='coolwarm', annot=True, fmt=".2f", cbar=False, ax=ax_saleprice_corr)
        ax_saleprice_corr.set_title('Correlación de Variables Numéricas con SalePrice')
        ax_saleprice_corr.tick_params(axis='y', rotation=0)
        st.pyplot(fig_saleprice_corr)

    else:
        st.warning("La columna 'SalePrice' no está disponible para calcular la correlación.")

elif viz_option == "Relación con SalePrice (Scatter plots)":
    st.subheader("Relación entre SalePrice y Variables Numéricas Clave")
    numeric_df_current = df_processed.select_dtypes(include=np.number)

    if 'SalePrice' in numeric_df_current.columns:
        correlation_with_saleprice = numeric_df_current.corr()['SalePrice'].sort_values(ascending=False)
        # Seleccionar las N variables más correlacionadas (excluyendo SalePrice)
        top_n = st.slider("Número de variables más correlacionadas a mostrar:", 3, 10, 5)
        top_correlated_cols = correlation_with_saleprice.head(top_n + 1).index.tolist()
        if 'SalePrice' in top_correlated_cols:
            top_correlated_cols.remove('SalePrice')

        st.write(f"Mostrando la relación de SalePrice con las {len(top_correlated_cols)} variables más correlacionadas:")

        # Calcular el número de filas y columnas para la cuadrícula de gráficos
        num_cols_plot = 3
        num_rows_plot = int(np.ceil(len(top_correlated_cols) / num_cols_plot))

        if len(top_correlated_cols) > 0:
            fig_scatter, axes_scatter = plt.subplots(num_rows_plot, num_cols_plot, figsize=(15, 5 * num_rows_plot))
            # Aplanar el array de axes si hay más de 1 fila
            axes_scatter = axes_scatter.flatten() if num_rows_plot > 1 else [axes_scatter]

            for i, col in enumerate(top_correlated_cols):
                 if col in numeric_df_current.columns:
                    sns.scatterplot(x=numeric_df_current[col], y=numeric_df_current['SalePrice'], ax=axes_scatter[i])
                    axes_scatter[i].set_title(f'SalePrice vs {col}')
                    axes_scatter[i].set_xlabel(col)
                    axes_scatter[i].set_ylabel('SalePrice')
                 else:
                     st.warning(f"La columna '{col}' no está disponible para scatter plot.")
                     axes_scatter[i].set_visible(False) # Ocultar el subplot vacío

            # Ocultar subplots vacíos si el número total de gráficos no llena la cuadrícula
            for j in range(i + 1, len(axes_scatter)):
                axes_scatter[j].set_visible(False)


            plt.tight_layout()
            st.pyplot(fig_scatter)
        else:
            st.info("No hay variables numéricas para graficar scatter plots con SalePrice.")

    else:
         st.warning("La columna 'SalePrice' no está disponible para crear scatter plots de correlación.")


elif viz_option == "Distribución de Variables Categóricas (Original)":
    st.subheader("Distribución de Variables Categóricas")
    # Para visualizar las variables categóricas con countplots, es mejor usar el df original
    # o al menos el df antes de One-Hot Encoding.
    # Asumiendo que df_processed mantiene las columnas categóricas originales antes de get_dummies
    # Opcional: Cargar el dataframe original si el df_processed ya ha sido OHEado
    try:
         # Intentar usar el df_processed antes de la codificación si está disponible
         # Si no, cargar el original.
         if not any(df_processed.select_dtypes(include='object').columns):
             df_original_for_viz = load_data() # Cargar el original de nuevo si df_processed no tiene categóricas

             # Aplicar solo la limpieza de nombres de columnas al original
             df_original_for_viz.columns = df_original_for_viz.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=True)

             # Aplicar imputación a categóricas si es necesario (para countplot)
             categorical_cols_original = df_original_for_viz.select_dtypes(include='object').columns.tolist()
             for col in categorical_cols_original:
                 if df_original_for_viz[col].isnull().any():
                    df_original_for_viz[col].fillna('Missing', inplace=True)

         else:
            # Si df_processed aún tiene columnas 'object', úsalo (asumiendo que ya fueron imputadas)
            df_original_for_viz = df_processed # Renombrar para claridad en esta sección


         categorical_cols_to_viz = df_original_for_viz.select_dtypes(include='object').columns.tolist()

         if not categorical_cols_to_viz:
             st.info("No hay columnas categóricas (tipo 'object') disponibles para graficar.")
         else:
            # Permitir al usuario seleccionar qué columnas categóricas visualizar
            selected_categorical_cols = st.multiselect("Selecciona variables categóricas para visualizar:",
                                                       categorical_cols_to_viz,
                                                       default=categorical_cols_to_viz[:5]) # Seleccionar las primeras 5 por defecto

            if selected_categorical_cols:
                # Determinar el número de filas y columnas para la cuadrícula de gráficos
                num_cols_cat_plot = 2
                num_rows_cat_plot = int(np.ceil(len(selected_categorical_cols) / num_cols_cat_plot))

                fig_cat, axes_cat = plt.subplots(num_rows_cat_plot, num_cols_cat_plot, figsize=(15, 5 * num_rows_cat_plot))
                axes_cat = axes_cat.flatten() if num_rows_cat_plot > 1 else [axes_cat]

                for i, col in enumerate(selected_categorical_cols):
                    if col in df_original_for_viz.columns and df_original_for_viz[col].dtype == 'object':
                        # Usar countplot para mostrar la frecuencia de cada categoría
                        sns.countplot(y=col, data=df_original_for_viz, order=df_original_for_viz[col].value_counts().index, ax=axes_cat[i])
                        axes_cat[i].set_title(f'Distribución de {col}')
                        axes_cat[i].set_xlabel('Frecuencia')
                        axes_cat[i].set_ylabel(col)
                    else:
                        st.warning(f"La columna '{col}' no es categórica o no existe en el DataFrame para gráfica de barras.")
                        axes_cat[i].set_visible(False) # Ocultar el subplot vacío

                # Ocultar subplots vacíos
                for j in range(i + 1, len(axes_cat)):
                    axes_cat[j].set_visible(False)

                plt.tight_layout()
                st.pyplot(fig_cat)
            else:
                st.info("Por favor, selecciona al menos una columna categórica para visualizar.")

    except FileNotFoundError:
        st.warning("No se pudo cargar el DataFrame original para visualizaciones categóricas.")

elif viz_option == "Distribución de Variables Numéricas":
    st.subheader("Distribución de Variables Numéricas")
    numeric_cols_to_viz = df_processed.select_dtypes(include=np.number).columns.tolist()
    # Excluir SalePrice para esta visualización si se muestra por separado
    if 'SalePrice' in numeric_cols_to_viz:
        numeric_cols_to_viz.remove('SalePrice')

    if not numeric_cols_to_viz:
        st.info("No hay columnas numéricas disponibles para graficar (excluyendo SalePrice).")
    else:
         selected_numeric_cols = st.multiselect("Selecciona variables numéricas para visualizar:",
                                               numeric_cols_to_viz,
                                               default=numeric_cols_to_viz[:5]) # Seleccionar las primeras 5 por defecto

         if selected_numeric_cols:
             # Determinar el número de filas y columnas para la cuadrícula de gráficos
             num_cols_num_plot = 3
             num_rows_num_plot = int(np.ceil(len(selected_numeric_cols) / num_cols_num_plot)) * 2 # Dos gráficos por columna

             fig_num, axes_num = plt.subplots(num_rows_num_plot, num_cols_num_plot, figsize=(15, 5 * num_rows_num_plot/2))
             axes_num = axes_num.flatten() if num_rows_num_plot > 1 else [axes_num]

             plot_index = 0
             for i, col in enumerate(selected_numeric_cols):
                 if col in df_processed.columns and df_processed[col].dtype in [np.number]:
                     # Histograma
                     sns.histplot(df_processed[col], kde=True, ax=axes_num[plot_index])
                     axes_num[plot_index].set_title(f'Distribución de {col}')
                     axes_num[plot_index].set_xlabel(col)
                     axes_num[plot_index].set_ylabel('Frecuencia')
                     plot_index += 1

                     # Boxplot
                     sns.boxplot(y=df_processed[col], ax=axes_num[plot_index])
                     axes_num[plot_index].set_title(f'Boxplot de {col}')
                     axes_num[plot_index].set_ylabel(col)
                     plot_index += 1
                 else:
                     st.warning(f"La columna '{col}' no es numérica o no existe en el DataFrame.")
                     # Ocultar los dos subplots correspondientes
                     axes_num[plot_index].set_visible(False)
                     axes_num[plot_index+1].set_visible(False)
                     plot_index += 2


             # Ocultar subplots vacíos
             for j in range(plot_index, len(axes_num)):
                 axes_num[j].set_visible(False)

             plt.tight_layout()
             st.pyplot(fig_num)
         else:
             st.info("Por favor, selecciona al menos una columna numérica para visualizar.")



# Separar datos para modelado (si aún no se ha hecho en preprocess_data)
if 'SalePrice' in df_processed.columns:
     X = df_processed.drop('SalePrice', axis=1)
     y = df_processed['SalePrice']
     # Asegurarse de que X es numérico después del get_dummies si preprocess_data no lo garantizó
     X = X.select_dtypes(include=np.number)
     # Asegurarse de que las filas coinciden (esto debería estar cubierto por la imputación sin NaNs)
     # X = X.dropna()
     # y = y.loc[X.index]
else:
    st.error("La columna 'SalePrice' es necesaria para el modelado.")
    st.stop()


# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # random_state para reproducibilidad

st.sidebar.subheader("Preparación de Datos")
st.sidebar.write(f"Dimensiones de X_train: {X_train.shape}")
st.sidebar.write(f"Dimensiones de X_test: {X_test.shape}")
st.sidebar.write(f"Dimensiones de y_train: {y_train.shape}")
st.sidebar.write(f"Dimensiones de y_test: {y_test.shape}")


# Configurar KFold para la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42) # shuffle=True y random_state para mezclar los datos consistentemente

# Definir métricas de evaluación
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    epsilon = 1e-8
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score, greater_is_better=True)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

scoring = {
    'mae': mae_scorer,
    'mse': mse_scorer,
    'r2': r2_scorer,
    'mape': mape_scorer
}
scoring_optimizer = {'mae': mae_scorer} # Usar MAE para la optimización


st.sidebar.subheader("Modelización")
selected_models = st.sidebar.multiselect("Selecciona modelos para entrenar y evaluar:",
                                         ['Linear Regression', 'Lasso', 'ElasticNet',
                                          'KernelRidge', 'SGDRegressor', 'BayesianRidge',
                                          'RandomForestRegressor', 'SVR'],
                                         default=['Linear Regression']) # Modelo por defecto

run_model_training = st.sidebar.button("Entrenar Modelos Seleccionados")

model_results = {}

if run_model_training and selected_models:
    st.subheader("Entrenamiento y Evaluación de Modelos")
    progress_bar = st.progress(0)
    status_text = st.empty()

    total_models = len(selected_models)
    model_count = 0

    for model_name in selected_models:
        model_count += 1
        status_text.text(f"Entrenando modelo: {model_name} ({model_count}/{total_models})")
        progress_bar.progress(model_count / total_models)

        start_time = time.time()

        # Inicializar modelo y definir espacios de búsqueda/parámetros si se usa optimización
        if model_name == 'Linear Regression':
            model = LinearRegression()
            # Sin optimización de hiperparámetros para LR base
            cv_results = cross_validate(model, X_train, y_train, cv=kf, scoring=scoring)
            best_estimator = model # No hay un "mejor estimador" de una búsqueda, solo el modelo entrenado

        elif model_name == 'Lasso':
             model = Lasso(random_state=42, max_iter=10000)
             # Usar BayesSearchCV para optimización (ejemplo)
             search_spaces = {'alpha': Real(0.001, 10, prior='log-uniform')}
             optimizer = BayesSearchCV(estimator=model, search_spaces=search_spaces,
                                      n_iter=50, scoring=scoring_optimizer, refit='mae',
                                      cv=kf, n_jobs=-1, random_state=42)
             optimizer.fit(X_train, y_train)
             best_estimator = optimizer.best_estimator_
             cv_results = cross_validate(best_estimator, X_train, y_train, cv=kf, scoring=scoring) # Evaluar el mejor con CV

        elif model_name == 'ElasticNet':
             model = ElasticNet(random_state=42, max_iter=10000)
             # Usar BayesSearchCV para optimización (ejemplo)
             search_spaces = {
                 'alpha': Real(0.001, 1, prior='log-uniform'),
                 'l1_ratio': Real(0, 1, prior='uniform')
             }
             optimizer = BayesSearchCV(estimator=model, search_spaces=search_spaces,
                                      n_iter=50, scoring=scoring_optimizer, refit='mae',
                                      cv=kf, n_jobs=-1, random_state=42)
             optimizer.fit(X_train, y_train)
             best_estimator = optimizer.best_estimator_
             cv_results = cross_validate(best_estimator, X_train, y_train, cv=kf, scoring=scoring) # Evaluar el mejor con CV


        elif model_name == 'KernelRidge':
             model = KernelRidge(kernel='rbf')
             # Usar BayesSearchCV para optimización (ejemplo)
             search_spaces = {
                 'alpha': Real(0.001, 1, prior='log-uniform'),
                 'gamma': Real(0.001, 1, prior='log-uniform')
             }
             optimizer = BayesSearchCV(estimator=model, search_spaces=search_spaces,
                                      n_iter=50, scoring=scoring_optimizer, refit='mae',
                                      cv=kf, n_jobs=-1, random_state=42)
             optimizer.fit(X_train, y_train)
             best_estimator = optimizer.best_estimator_
             cv_results = cross_validate(best_estimator, X_train, y_train, cv=kf, scoring=scoring) # Evaluar el mejor con CV

        elif model_name == 'SGDRegressor':
             model = SGDRegressor(learning_rate='invscaling', early_stopping=True,
                                 random_state=42, max_iter=10000)
             # Usar BayesSearchCV para optimización (ejemplo)
             search_spaces = {'alpha': Real(0.0001, 0.01, prior='log-uniform')}
             optimizer = BayesSearchCV(estimator=model, search_spaces=search_spaces,
                                      n_iter=50, scoring=scoring_optimizer, refit='mae',
                                      cv=kf, n_jobs=-1, random_state=42)
             optimizer.fit(X_train, y_train)
             best_estimator = optimizer.best_estimator_
             cv_results = cross_validate(best_estimator, X_train, y_train, cv=kf, scoring=scoring) # Evaluar el mejor con CV

        elif model_name == 'BayesianRidge':
            model = BayesianRidge()
             # Sin optimización de hiperparámetros para BR base
            cv_results = cross_validate(model, X_train, y_train, cv=kf, scoring=scoring)
            best_estimator = model # No hay un "mejor estimador" de una búsqueda, solo el modelo entrenado


        elif model_name == 'RandomForestRegressor':
             model = RandomForestRegressor(random_state=42, n_jobs=-1)
             # Usar BayesSearchCV para optimización (ejemplo)
             search_spaces = {
                 'n_estimators': Integer(50, 150),
                 'max_depth': Integer(5, 15),
                 'min_samples_split': Integer(2, 10)
             }
             optimizer = BayesSearchCV(estimator=model, search_spaces=search_spaces,
                                      n_iter=10, # Menos iteraciones para acelerar
                                      scoring=scoring_optimizer, refit='mae',
                                      cv=kf, n_jobs=-1, random_state=42)
             optimizer.fit(X_train, y_train)
             best_estimator = optimizer.best_estimator_
             cv_results = cross_validate(best_estimator, X_train, y_train, cv=kf, scoring=scoring) # Evaluar el mejor con CV

        elif model_name == 'SVR':
             model = SVR(kernel='rbf')
             # Usar BayesSearchCV para optimización (ejemplo)
             search_spaces = {
                 'C': Real(1e-1, 1e3, prior='log-uniform'),
                 'epsilon': Real(1e-3, 1e0, prior='uniform'),
                 'gamma': Real(1e-4, 1e0, prior='log-uniform')
             }
             optimizer = BayesSearchCV(estimator=model, search_spaces=search_spaces,
                                      n_iter=50, # Menos iteraciones para acelerar
                                      scoring=scoring_optimizer, refit='mae',
                                      cv=kf, n_jobs=-1, random_state=42)
             optimizer.fit(X_train, y_train)
             best_estimator = optimizer.best_estimator_
             cv_results = cross_validate(best_estimator, X_train, y_train, cv=kf, scoring=scoring) # Evaluar el mejor con CV

        end_time = time.time()
        elapsed_time = end_time - start_time

        # Guardar resultados
        # MAE, MSE, MAPE son negativas por make_scorer, tomar el valor absoluto
        model_results[model_name] = {
            'MAE': -cv_results['test_mae'].mean(),
            'MSE': -cv_results['test_mse'].mean(),
            'R2': cv_results['test_r2'].mean(),
            'MAPE': -cv_results['test_mape'].mean(),
            'Time (s)': elapsed_time,
            'Best Estimator': best_estimator # Guardar el mejor estimador encontrado
        }
        st.write(f"--- Resultados para {model_name} ---")
        st.write(f"  MAE Promedio: {model_results[model_name]['MAE']:.4f}")
        st.write(f"  MSE Promedio: {model_results[model_name]['MSE']:.4f}")
        st.write(f"  R2 Promedio: {model_results[model_name]['R2']:.4f}")
        st.write(f"  MAPE Promedio: {model_results[model_name]['MAPE']:.4f}%")
        st.write(f"  Tiempo de Entrenamiento: {model_results[model_name]['Time (s)']:.2f} segundos")
        if model_name in ['Lasso', 'ElasticNet', 'KernelRidge', 'SGDRegressor', 'RandomForestRegressor', 'SVR']:
             st.write(f"  Mejores Hiperparámetros (Optimizados): {optimizer.best_params_}")


    status_text.text("Entrenamiento completado!")
    progress_bar.empty() # Ocultar la barra de progreso


    st.subheader("Comparación de Modelos")
    results_df = pd.DataFrame.from_dict(model_results, orient='index')

    # Mostrar tabla de resultados
    st.dataframe(results_df)

    # Generar gráficas de comparación
    metrics_to_plot = ['MAE', 'MSE', 'R2', 'MAPE']
    for metric in metrics_to_plot:
        fig_comp, ax_comp = plt.subplots(figsize=(12, 6))
        ascending = True if metric in ['MAE', 'MSE', 'MAPE'] else False
        results_df_sorted = results_df.sort_values(by=metric, ascending=ascending)

        sns.barplot(x=results_df_sorted.index, y=metric, data=results_df_sorted, ax=ax_comp)
        ax_comp.set_title(f'Comparación de Modelos por {metric}', fontsize=16)
        ax_comp.set_xlabel("Modelo", fontsize=12)
        ax_comp.set_ylabel(metric, fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        st.pyplot(fig_comp)

    # Visualizar importancia de características para modelos aplicables
    st.subheader("Importancia de Características (para modelos aplicables)")
    models_with_importance = [m for m in selected_models if m in ['Linear Regression', 'Lasso', 'ElasticNet', 'SGDRegressor', 'BayesianRidge', 'RandomForestRegressor']]

    if models_with_importance:
        selected_model_importance = st.selectbox("Selecciona un modelo para ver la importancia de características:",
                                                models_with_importance)

        if selected_model_importance:
            best_model_importance = model_results[selected_model_importance]['Best Estimator']

            if hasattr(best_model_importance, 'coef_'): # Linear models
                 if isinstance(X_train, pd.DataFrame):
                     coefficients = pd.Series(best_model_importance.coef_, index=X_train.columns)
                     sorted_coefficients = coefficients.abs().sort_values(ascending=False)

                     fig_coef, ax_coef = plt.subplots(figsize=(10, min(20, len(sorted_coefficients) * 0.3)))
                     sns.barplot(x=sorted_coefficients.values, y=sorted_coefficients.index, ax=ax_coef)
                     ax_coef.set_title(f'Importancia de Características (Coeficientes Absolutos) - {selected_model_importance}', fontsize=14)
                     ax_coef.set_xlabel('Magnitud del Coeficiente', fontsize=12)
                     ax_coef.set_ylabel('Característica', fontsize=12)
                     plt.tight_layout()
                     st.pyplot(fig_coef)
                 else:
                      st.warning("X_train no es un DataFrame. No se puede mostrar la importancia de características basada en coeficientes.")


            elif hasattr(best_model_importance, 'feature_importances_'): # Tree-based models
                 if isinstance(X_train, pd.DataFrame):
                     importances = pd.Series(best_model_importance.feature_importances_, index=X_train.columns)
                     sorted_importances = importances.sort_values(ascending=False)

                     fig_feat_imp, ax_feat_imp = plt.subplots(figsize=(10, min(20, len(sorted_importances) * 0.3)))
                     sns.barplot(x=sorted_importances.values, y=sorted_importances.index, ax=ax_feat_imp)
                     ax_feat_imp.set_title(f'Importancia de Características - {selected_model_importance}', fontsize=14)
                     ax_feat_imp.set_xlabel('Importancia', fontsize=12)
                     ax_feat_imp.set_ylabel('Característica', fontsize=12)
                     plt.tight_layout()
                     st.pyplot(fig_feat_imp)
                 else:
                     st.warning("X_train no es un DataFrame. No se puede mostrar la importancia de características basada en feature_importances.")
            else:
                st.info(f"El modelo {selected_model_importance} no proporciona una medida estándar de importancia de características (coeficientes o feature_importances).")
    else:
        st.info("Selecciona modelos lineales o basados en árboles para visualizar la importancia de características.")

elif run_model_training and not selected_models:
    st.warning("Por favor, selecciona al menos un modelo para entrenar.")


In [ ]:
# prompt: Para los modelos de random, kernel y elasticnet crea Gráficas de barras para comparar MAE, MSE, R2 y MAPE con plt.bar(model_names, mae_values) y st.pyplot().
# Una tabla con las métricas usando st.dataframe(pd.DataFrame(model_results).T).

import matplotlib.pyplot as plt
import streamlit as st
import pandas as pd
import seaborn as sns

# Ensure the necessary metric variables from previous steps are available.
# Assuming cv_results_bayes_krr, cv_results_bayes_en, cv_results_best_rf and their corresponding
# mean metric values (cv_mae_bayes_krr.mean(), cv_mse_bayes_krr.mean(), etc.) are defined in
# the preceding code blocks.

# --- Comparison Graphs and Table for KernelRidge, ElasticNet, RandomForestRegressor ---

# Define the model names and their corresponding metric values
model_names = ['KernelRidge (Bayes)', 'ElasticNet (Bayes)', 'RandomForestRegressor (Bayes)']

# Collect the metric values for the selected models
mae_values = [
    -cv_results_bayes_best_krr['test_mae'].mean(),
    -cv_results_bayes_best_en['test_mae'].mean(),
    -cv_results_best_rf['test_mae'].mean()
]

mse_values = [
    -cv_results_bayes_best_krr['test_mse'].mean(),
    -cv_results_bayes_best_en['test_mse'].mean(),
    -cv_results_best_rf['test_mse'].mean()
]

r2_values = [
    cv_results_bayes_best_krr['test_r2'].mean(),
    cv_results_bayes_best_en['test_r2'].mean(),
    cv_results_best_rf['test_r2'].mean()
]

mape_values = [
    -cv_results_bayes_best_krr['test_mape'].mean(),
    -cv_results_bayes_best_en['test_mape'].mean(),
    -cv_results_best_rf['test_mape'].mean()
]


# Create a dictionary for the results table
model_comparison_results = {
    'KernelRidge (Bayes)': {
        'MAE': mae_values[0],
        'MSE': mse_values[0],
        'R2': r2_values[0],
        'MAPE': mape_values[0]
    },
    'ElasticNet (Bayes)': {
        'MAE': mae_values[1],
        'MSE': mse_values[1],
        'R2': r2_values[1],
        'MAPE': mape_values[1]
    },
    'RandomForestRegressor (Bayes)': {
        'MAE': mae_values[2],
        'MSE': mse_values[2],
        'R2': r2_values[2],
        'MAPE': mape_values[2]
    }
}

# Create a DataFrame for the table
model_comparison_df = pd.DataFrame(model_comparison_results).T

# Display the comparison table using Streamlit
st.subheader("Comparación de Métricas (KernelRidge, ElasticNet, RandomForestRegressor)")
st.dataframe(model_comparison_df)

# Create and display bar charts for each metric using Streamlit
metrics_to_plot = ['MAE', 'MSE', 'R2', 'MAPE']
y_labels = {'MAE': 'MAE', 'MSE': 'MSE', 'R2': 'R2 Score', 'MAPE': 'MAPE (%)'}
titles = {
    'MAE': 'Comparación de MAE',
    'MSE': 'Comparación de MSE',
    'R2': 'Comparación de R2 Score',
    'MAPE': 'Comparación de MAPE'
}
ascending = {'MAE': True, 'MSE': True, 'R2': False, 'MAPE': True} # Lower is better for MAE, MSE, MAPE; Higher is better for R2

for metric in metrics_to_plot:
    fig, ax = plt.subplots(figsize=(10, 6))
    # Sort values for better visualization (optional but recommended)
    sorted_df = model_comparison_df.sort_values(by=metric, ascending=ascending[metric])
    ax.bar(sorted_df.index, sorted_df[metric])
    ax.set_title(titles[metric], fontsize=16)
    ax.set_xlabel("Modelo", fontsize=12)
    ax.set_ylabel(y_labels[metric], fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
# prompt: Para los modelos de random, kernel y elasticnet crea Gráficas de barras para comparar MAE, MSE, R2 y MAPE con plt.bar(model_names, mae_values) y st.pyplot().
# Una tabla con las métricas usando st.dataframe(pd.DataFrame(model_results).T).

import matplotlib.pyplot as plt
import streamlit as st
import pandas as pd
import seaborn as sns

# Ensure the necessary metric variables from previous steps are available.
# Assuming cv_results_bayes_krr, cv_results_bayes_en, cv_results_best_rf and their corresponding
# mean metric values (cv_mae_bayes_krr.mean(), cv_mse_bayes_krr.mean(), etc.) are defined in
# the preceding code blocks.

# --- Comparison Graphs and Table for KernelRidge, ElasticNet, RandomForestRegressor ---

# Define the model names and their corresponding metric values
model_names = ['KernelRidge (Bayes)', 'ElasticNet (Bayes)', 'RandomForestRegressor (Bayes)']

# Collect the metric values for the selected models
mae_values = [
    -cv_results_bayes_best_krr['test_mae'].mean(),
    -cv_results_bayes_best_en['test_mae'].mean(),
    -cv_results_best_rf['test_mae'].mean()
]

mse_values = [
    -cv_results_bayes_best_krr['test_mse'].mean(),
    -cv_results_bayes_best_en['test_mse'].mean(),
    -cv_results_best_rf['test_mse'].mean()
]

r2_values = [
    cv_results_bayes_best_krr['test_r2'].mean(),
    cv_results_bayes_best_en['test_r2'].mean(),
    cv_results_best_rf['test_r2'].mean()
]

mape_values = [
    -cv_results_bayes_best_krr['test_mape'].mean(),
    -cv_results_bayes_best_en['test_mape'].mean(),
    -cv_results_best_rf['test_mape'].mean()
]


# Create a dictionary for the results table
model_comparison_results = {
    'KernelRidge (Bayes)': {
        'MAE': mae_values[0],
        'MSE': mse_values[0],
        'R2': r2_values[0],
        'MAPE': mape_values[0]
    },
    'ElasticNet (Bayes)': {
        'MAE': mae_values[1],
        'MSE': mse_values[1],
        'R2': r2_values[1],
        'MAPE': mape_values[1]
    },
    'RandomForestRegressor (Bayes)': {
        'MAE': mae_values[2],
        'MSE': mse_values[2],
        'R2': r2_values[2],
        'MAPE': mape_values[2]
    }
}

# Create a DataFrame for the table
model_comparison_df = pd.DataFrame(model_comparison_results).T

# Display the comparison table using Streamlit
st.subheader("Comparación de Métricas (KernelRidge, ElasticNet, RandomForestRegressor)")
st.dataframe(model_comparison_df)

# Create and display bar charts for each metric using Streamlit
metrics_to_plot = ['MAE', 'MSE', 'R2', 'MAPE']
y_labels = {'MAE': 'MAE', 'MSE': 'MSE', 'R2': 'R2 Score', 'MAPE': 'MAPE (%)'}
titles = {
    'MAE': 'Comparación de MAE',
    'MSE': 'Comparación de MSE',
    'R2': 'Comparación de R2 Score',
    'MAPE': 'Comparación de MAPE'
}
ascending = {'MAE': True, 'MSE': True, 'R2': False, 'MAPE': True} # Lower is better for MAE, MSE, MAPE; Higher is better for R2

for metric in metrics_to_plot:
    fig, ax = plt.subplots(figsize=(10, 6))
    # Sort values for better visualization (optional but recommended)
    sorted_df = model_comparison_df.sort_values(by=metric, ascending=ascending[metric])
    ax.bar(sorted_df.index, sorted_df[metric])
    ax.set_title(titles[metric], fontsize=16)
    ax.set_xlabel("Modelo", fontsize=12)
    ax.set_ylabel(y_labels[metric], fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    st.pyplot(fig)


In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run TAM.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL